<a href="https://colab.research.google.com/github/nurrizkyimani/nlp-clickbait-transfers/blob/main/NEW_Dec_15_512_scheduler_mix_adam_sgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip3 install fse
!pip3 install pytorch-nlp

# conda install -c anaconda nltk
#  conda install -c anaconda pandas
# conda create -n sklearn-env -c conda-forge scikit-learn
# conda activate sklearn-env
# conda install pytorch torchvision torchaudio -c pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.7 MB/s 
     |████████████████████████████████| 24.1 MB 1.1 MB/s 
     |████████████████████████████████| 56.8 MB 3.1 MB/s 
     |████████████████████████████████| 182 kB 68.3 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for fse: filename=fse-1.0.0-cp38-cp38-linux_x86_64.whl size=256586 sha256=6ea996e5f3740d66603ac6232949db0bad67ee449ea8902155eb85239d39f302
  Stored in directory: /root/.cache/pip/wheels/c8/6b/c6/8fd0c371410cf2f70f57dc53f127423d443dea6238b8764a0e
Successfully built fse
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.6 MB/s 


# Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import re
import string
import collections
import fse
import random

import random
import math
import time

import itertools
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchnlp.metrics import get_moses_multi_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)

import nltk
nltk.download('punkt')

import nltk
from nltk.util import ngrams
nltk.download('punkt')
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

# Any results you write to the current directory are saved as output.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# !wget "https://storage.googleapis.com/babert-pretraining/IndoNLU_finals/models/fasttext/fasttext.4B.id.300.epoch5.uncased.bin" -P "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer"

# Importing Dataset GDrive

In [ ]:
# import dataset from google drive
path = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/annotated/combined/csv/"
all_agree_path = path + "main.csv"
all_agree_pd = pd.read_csv(all_agree_path)
df_clickbait = all_agree_pd[all_agree_pd['label'] == "clickbait"]
df_nonclickbait = all_agree_pd[all_agree_pd['label'] == "non-clickbait"]
df_both = pd.concat((df_clickbait, df_nonclickbait), ignore_index=True)

df_both_copy = df_both.copy()
df_both_copy['label'] = df_both_copy['label'].astype('category')
df_both_copy.head(-1)

,title,label,label_score
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,1
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",clickbait,1
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,clickbait,1
3,Detik-detik Lima Kendaraan Alami Kecelakaan Be...,clickbait,1
4,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,clickbait,1
...,...,...,...
14994,"Diperankan Bastian Bintang, ANTV Bakal Hadirka...",non-clickbait,0
14995,Gubernur Tetapkan Riau Darurat Pencemaran Udar...,non-clickbait,0
14996,"Asap Karhutla Riau Mulai Merambah Ke Nias, BM...",non-clickbait,0
14997,"Tolak RUU Pertanahan, Ribuan Petani Siap Gelar...",non-clickbait,0


# Importing dataset

In [ ]:
# # Import the csv
# all_agree_path = "2022_thesis_styletransfer/annotated/combined/csv/main.csv"
# all_agree_pd = pd.read_csv(all_agree_path)
# df_clickbait = all_agree_pd[all_agree_pd['label'] == "clizckbait"]
# df_nonclickbait = all_agree_pd[all_agree_pd['label'] == "non-clickbait"]
# df_both = pd.concat((df_clickbait, df_nonclickbait), ignore_index=True)

# df_both_copy = df_both.copy()
# df_both_copy['label'] = df_both_copy['label'].astype('category')
# df_both_copy.head(-1)

In [ ]:
df_both_label = df_both_copy.drop('label_score', axis=1)
df_both = df_both_copy.drop('label', axis=1)
df_both.head(-5)

,title,label_score
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,1
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",1
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,1
3,Detik-detik Lima Kendaraan Alami Kecelakaan Be...,1
4,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,1
...,...,...
14990,"Wamena Papua Kembali Membara, Kantor Bupati Di...",0
14991,BMKG Angkat Bicara Soal Langit Merah Jambi,0
14992,Densus 88 Tangkap Teroris Jaringan ISIS di Jak...,0
14993,Operasional Bandara Wamena Dihentikan Akibat R...,0


In [ ]:
len(df_both)

15000

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df_both_copy['label_encoding'] = labelencoder.fit_transform(df_both_copy['label'])
df_encode_clean = df_both_copy.copy()
df_encode_clean = df_both = df_both_copy.drop(['label_score'], axis=1)
df_encode_clean.head()

,title,label,label_encoding
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,0
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",clickbait,0
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,clickbait,0
3,Detik-detik Lima Kendaraan Alami Kecelakaan Be...,clickbait,0
4,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,clickbait,0


# Preprocessing : Lower Casing

In [ ]:
# preprocessing : lower casing
df_encode_clean['title'] = df_encode_clean['title'].str.lower()
df_encode_clean.title

0        viral! driver ojol di bekasi antar pesanan mak...
1        ada motor nyangkut di atas bambu di sleman, ko...
2             pesan gamblang poyuono menolak revisi uu kpk
3        detik-detik lima kendaraan alami kecelakaan be...
4        kocak! maling di rumah mewah jakut terekam cct...
                               ...                        
14995    gubernur tetapkan riau darurat pencemaran udar...
14996     asap karhutla riau mulai merambah ke nias, bm...
14997    tolak ruu pertanahan, ribuan petani siap gelar...
14998    beredar isu internet papua diblokir lagi, telk...
14999    emmy awards 2019: peter dinklage cetak rekor b...
Name: title, Length: 15000, dtype: object

Convert the titl into lower case for all label

In [ ]:
# Add space between punctuation
df_encode_clean_t =  df_encode_clean.copy()
df_encode_clean_t['title'] = df_encode_clean_t['title'].apply(lambda y: " ".join((re.sub(r'([!/?/./,/%/\'/\"/-/:])', lambda x: ' ' + x.group()+' ' , y)).split()) , 1)
df_encode_clean_t

,title,label,label_encoding
0,viral ! driver ojol di bekasi antar pesanan ma...,clickbait,0
1,"ada motor nyangkut di atas bambu di sleman , k...",clickbait,0
2,pesan gamblang poyuono menolak revisi uu kpk,clickbait,0
3,detik-detik lima kendaraan alami kecelakaan be...,clickbait,0
4,kocak ! maling di rumah mewah jakut terekam cc...,clickbait,0
...,...,...,...
14995,gubernur tetapkan riau darurat pencemaran udar...,non-clickbait,1
14996,"asap karhutla riau mulai merambah ke nias , bm...",non-clickbait,1
14997,"tolak ruu pertanahan , ribuan petani siap gela...",non-clickbait,1
14998,"beredar isu internet papua diblokir lagi , tel...",non-clickbait,1


In [ ]:
# separate between clickbait clean and nonclickbait clean
df_clickbait_clean = df_encode_clean_t[df_encode_clean_t['label'] == "clickbait"]
df_nonclickbait_clean = df_encode_clean_t[df_encode_clean_t['label'] == "non-clickbait"]

In [ ]:
# create a list form the combination of df_clickbait and nonclickbait

d_both = pd.concat((df_clickbait_clean, df_nonclickbait_clean ), ignore_index=True)
d_both = d_both.drop(['label', 'label_encoding'], axis=1)
d_both = d_both.values.tolist()
d_both = list(itertools.chain.from_iterable(d_both))

In [ ]:
d_both_clean = pd.concat((df_clickbait_clean, df_nonclickbait_clean ), ignore_index=True)
d_both_clean = d_both_clean.drop(['label'], axis=1)
d_both_clean

,title,label_encoding
0,viral ! driver ojol di bekasi antar pesanan ma...,0
1,"ada motor nyangkut di atas bambu di sleman , k...",0
2,pesan gamblang poyuono menolak revisi uu kpk,0
3,detik-detik lima kendaraan alami kecelakaan be...,0
4,kocak ! maling di rumah mewah jakut terekam cc...,0
...,...,...
14995,gubernur tetapkan riau darurat pencemaran udar...,1
14996,"asap karhutla riau mulai merambah ke nias , bm...",1
14997,"tolak ruu pertanahan , ribuan petani siap gela...",1
14998,"beredar isu internet papua diblokir lagi , tel...",1


In [ ]:
d_both

['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda',
 'ada motor nyangkut di atas bambu di sleman , kok bisa ?',
 'pesan gamblang poyuono menolak revisi uu kpk',
 'detik-detik lima kendaraan alami kecelakaan beruntun di cipularang',
 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur',
 'hampir seluas jabar , ini tantangan sintang siapkan generasi emas',
 'viral video diduga baku tembak di sleman , ini kata polisi',
 'waspada ! ada penipuan catut pertamina , korbannya rugi puluhan juta',
 'jaksa beberkan senpi-peluru tajam pembelian kivlan zen cs , ini rinciannya',
 'sering quality time bersama keluarga ? ternyata ini 3 manfaatnya !',
 'menteri jokowi 55 % profesional , ini kandidatnya ?',
 'sebut pemilihan capim kpk cacat yuridis , ini saran samad ke dpr',
 "pria misterius berjubah putih viral di sumut , ingatkan ' tuhan murka '",
 'video porno di sumedang disebar pemeran pria , apa motifnya ?',
 'terungkap ! video porno di sumedang diperankan p

In [ ]:
# create list of clickbait and nonclickbait headline

df_clickbait_t = df_clickbait_clean.drop(['label', 'label_encoding'], axis=1).values.tolist()
clickbait_l_c = list(itertools.chain.from_iterable(df_clickbait_t))

df_nonclickbait_t = df_nonclickbait_clean.drop(['label', 'label_encoding'], axis=1).values.tolist()
nonclickbait_l_c = list(itertools.chain.from_iterable(df_nonclickbait_t))

print(nonclickbait_l_c[0:5])
print(clickbait_l_c[0:5])

['masuk radar pilwalkot medan , menantu jokowi bertemu dpw nasdem sumut', 'malaysia sudutkan ri : isu kabut asap hingga invasi babi', 'kemensos salurkan rp 7 , 3 m bagi korban kerusuhan sosial di papua', 'terkait mayat bayi mengenaskan di tangerang , seorang pria ditangkap polisi', 'uang rp 1 , 6 m yang raib di parkiran kantor gubernur sumut untuk honor tapd']
['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda', 'ada motor nyangkut di atas bambu di sleman , kok bisa ?', 'pesan gamblang poyuono menolak revisi uu kpk', 'detik-detik lima kendaraan alami kecelakaan beruntun di cipularang', 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur']


# Feature Extraction: Ngram [100% DONE]

In [ ]:
#these are methods that will become useful when extracting attribute markers
#why do we need all this? well... that's like 5 hours of debugging...
def flatten(foo):
    return list(_flatten(foo))

def _flatten(foo):
    for x in foo:
        if isinstance(x, collections.Iterable) and not isinstance(x, str):
            for y in _flatten(x):
                yield y
        else:
            yield x

def array_to_string(a):
    return ' '.join(flatten(a))

def is_in_string_array(elements, original): #deprecated, does not take into account sequence order
    return np.isin(array_to_string(elements).split(), array_to_string(original).split()).any()
# no usage in the func
def insert_string(string, inserted_string, index):
    return string[:index] + inserted_string + string[index:]

# modified from https://stackoverflow.com/questions/41752946/replacing-a-character-from-a-certain-index
def replace_string(s, newstring, index, nofail=False):
    # raise an error if index is outside of the string
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string. index:" + index)

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + len(newstring):]

In [ ]:
# creating function for making ngram from list of headline
def ngram_maker_v2(list_sentences, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    res_ngram = {length : [] for length in lengths}

    for leng in lengths:
        for sentence in list_sentences:
            temp_set = set()
            n_grams = ngrams(nltk.word_tokenize(sentence), leng)

            for grams in n_grams:
                res = ' '.join(grams)
                temp_set.add(res)

            for key in temp_set:
                split_t = tuple(key.split(" "))
                res_ngram[leng].append(split_t)

    return res_ngram

In [ ]:
# create a function that count the ngram occurance
def counter_for_ngram(ngram_dict, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    ngram_count_res = {length : [] for length in lengths}

    for key in ngram_dict:
        ng_count = Counter(ngram_dict[key])
        ngram_count_res[key] = ng_count

    return ngram_count_res

In [ ]:
param_span_t = 3
clickbait_ngrams_v2= ngram_maker_v2(list_sentences=clickbait_l_c, min_length=1, max_length=param_span_t)
clickbait_ngram_count_v2 = counter_for_ngram(ngram_dict=clickbait_ngrams_v2,min_length=1, max_length=param_span_t)

non_clickbait_ngrams_v2= ngram_maker_v2(list_sentences=nonclickbait_l_c, min_length=1, max_length=param_span_t)
non_clickbait_ngram_count_v2 = counter_for_ngram(ngram_dict=non_clickbait_ngrams_v2,min_length=1, max_length=param_span_t)

In [ ]:
# creat function for get ngram from the word in the one sentence reference only
def ngram_from_sentence_v3(ori_sentence, min_length, max_length):
    sent_in_arr = [ori_sentence]
    res_ngram_sentence = []
    stc_ngram_only = ngram_maker_v2(list_sentences=sent_in_arr, min_length=min_length, max_length=max_length)
    return stc_ngram_only

def count_ngram_sentence_onlyV3(ngram_dictionary, ngram_from_sentence, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    ngram_compare_arr = []

    for length in lengths:
        for split in ngram_from_sentence[length]:
            ngram_compare_arr.append([split, ngram_dictionary[length][split]])

    return np.array(ngram_compare_arr)

In [ ]:
# create function for get the attribute of the sentence

smoothing_parameter = 1

def array_to_string(a):
    return ' '.join(flatten(a))

# the word is in the final array;
def is_in_string_array_v2(elements, original):
    return np.isin(array_to_string(elements).split(), array_to_string(original).split()).any()

# the the attribute based on the sentence, with min_length, max_length and the reference clickbait/nonclickbait dictinoary
def get_attribute_sentence(sentence, min_length, max_length, ref_bait, param_threshold):
    # ngram sentence as reference
    sentence_ngram_ref = ngram_from_sentence_v3(ori_sentence=sentence, min_length=min_length, max_length=max_length)

    # count the clickbait sentence, clickbait ngram count as based reference
    counts_sentence_clickbait = count_ngram_sentence_onlyV3(clickbait_ngram_count_v2,sentence_ngram_ref,min_length,max_length)

    # print(counts_sentence_clickbait)

    # count the non-clickbait sentence, non-clickbait ngram count as based reference
    counts_sentence_nonclickbait = count_ngram_sentence_onlyV3(non_clickbait_ngram_count_v2,sentence_ngram_ref,min_length,max_length)

    importances = 0

    csn_int = counts_sentence_nonclickbait[:,1]
    csn_ngram = counts_sentence_nonclickbait[:,0]

    csc_int = counts_sentence_clickbait[:,1]
    csc_ngram = counts_sentence_clickbait[:,0]

    if ref_bait == "to-clickbait":
        top = (csc_int + smoothing_parameter)
        bottom = (csn_int + smoothing_parameter)
        importances = (top/bottom)
    elif ref_bait == "to-non-clickbait":
        top = (csn_int+ smoothing_parameter)
        bottom = (csc_int + smoothing_parameter)
        importances = (top/bottom)

    # print(counts_sentence_clickbait)

    de = counts_sentence_clickbait[:, 0]
    # print("impot: ", importances)

    # print(" de: ", de)
    importances_t = np.vstack((importances, de)).T

    # print("importances_t", importances_t)

    attribute_words = []
    for importance in importances_t:
        if importance[0] > param_threshold and not is_in_string_array_v2(importance[1], attribute_words):
            # print(importance[1])
            attribute_words.append(' '.join(importance[1]))

    return attribute_words

In [ ]:
# run the get attribute sentence
test11 = get_attribute_sentence("viral ini ! bj habibie meninggal", 1, 2, "to-clickbait", 4 )

<ipython-input-21-3f72f6d89c87>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)
<ipython-input-17-3795b38cea96>:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(x, collections.Iterable) and not isinstance(x, str):


In [ ]:
def separate(sentence, style_src, param_threshold):
    attributes = get_attribute_sentence(sentence, 1, 2, "to-clickbait", param_threshold )
    c = sentence

    replace_indexes = []

    for a in attributes:
        replace_index = -1
        replace_index = c.find(a)
        replace_indexes.append(replace_index)
        c = c.replace(a, " "*len(a))

    if len(attributes) == 0:
        return {'c': c, 'a': [], 'i': [], 's': sentence}

    replace_indexes, attributes = zip(*sorted(zip(replace_indexes, attributes)))
    return {'c': c, 'a': attributes, 'i': replace_indexes, 's': sentence}

def get_c(sentence, style, param_threshold):
    return re.sub(' +', ' ', separate(sentence, style, param_threshold)['c'])

def get_a(sentence, style, param_threshold):
    a = separate(sentence, style, param_threshold)['a']
    if len(a) > 0:
        return ' '.join(a)
    else:
        return ""

# Preprocessed : TFIDF, Retrieve Function [100% DONE]

to get similar sentece in the clickbait headline list

In [ ]:
# 3 of all headline list
# d_both
# nonclickbait_l_c
# clickbait_l_c

In [ ]:
# import all needed librarty
tfidf_vectorizer=TfidfVectorizer(use_idf=True, stop_words=None)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(d_both)
dict_idf = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))

/Users/nurrizkyimani/.conda/envs/workspace/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
param_backoff_limit = 3
def get_overlap(a, b):
    #     print(a, b)
    a_counter = collections.Counter(a.split())
    b_counter = collections.Counter(b.split())
    overlap = a_counter & b_counter
    return overlap

def get_weighted_overlap(a, b):
    overlap = get_overlap(a, b)
    a_counter = collections.Counter(a.split())
    #calculate
    weighted_overlap = 0
    for word in overlap:

        word_tf = a_counter[word]#/len(a.split()) -> commented out cause division by constant value doesn't matter

        get_idf = dict_idf.get(word)
        word_idf = 1 if get_idf == None else get_idf #get rid of error when idf not in dict

        word_tfidf = word_tf*word_idf
        weighted_overlap+=overlap[word]*word_tfidf

    return weighted_overlap

# 0 = nonclickbait_l_c
# 1 = clickbait_l_c

def get_similar_sentence_v2(sentence, style_src):
    opposite_dataset = clickbait_l_c if style_src else nonclickbait_l_c

    closest_sentence = ""
    highest_overlap = 0
    previous_sentences = []

    for sentence_b in opposite_dataset:
        weighted_overlap = get_weighted_overlap(sentence, sentence_b)
        if weighted_overlap > highest_overlap and sentence_b not in previous_sentences:
            highest_overlap = weighted_overlap
            closest_sentence = sentence_b

    return closest_sentence

def get_similar_sentence(sentence, style_src):

    opposite_dataset = clickbait_l_c if style_src else nonclickbait_l_c
    highest_overlap = 0
    closest_sentence = ""
    num_markers = 0
    previous_sentences = []
    backoff_count = 0

    min_attribute_markers=len(get_attribute_sentence(sentence, 1, 2, "to-non-clickbait", 5 ))

    while(num_markers < min_attribute_markers and backoff_count < param_backoff_limit):
        for sentence_b in opposite_dataset:
            weighted_overlap = get_weighted_overlap(sentence, sentence_b)
            if weighted_overlap > highest_overlap and sentence_b not in previous_sentences:
                highest_overlap = weighted_overlap
                closest_sentence = sentence_b
        highest_overlap = 0
        backoff_count += 1
        previous_sentences.append(closest_sentence)
        num_markers = len((get_attribute_sentence(closest_sentence, 1, 2, "to-clickbait", 5 )))

    return closest_sentence

In [ ]:
# Retrieve using tfidf
def retrieve_v2(sentence, style_ret_src):
    style_src = 0
    if(style_ret_src == "to-clickbait"):
        style_src = 1

    similar_stc =  get_similar_sentence_v2(sentence=sentence, style_src=style_src)
    print("v2 stc: ", similar_stc)
    return separate(sentence=similar_stc, style_src= "to-clickbait", param_threshold=5)

# Retrieve using tfidf
def retrieve(sentence, style_ret_src):
    style_src = 0
    if(style_ret_src == "to-clickbait"):
        style_src = 1

    similar_stc =  get_similar_sentence(sentence=sentence, style_src=style_src)
    print("v1 stc: ", similar_stc)
    return separate(sentence=similar_stc, style_src= "to-clickbait", param_threshold=5)

In [ ]:
stc_test_retrieve = "tim jibom disposal bahan peledak di rumah terduga teroris di jakut"
retrieve(stc_test_retrieve, "to-clickbait")
# retrieve_v2(stc_test_retrieve,  "to-clickbait")

v1 stc:  duarr ! bahan peledak terduga teroris jakut didisposal , warga tepuk tangan


/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_46291/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)


{'c': 'duarr   bahan peledak terduga teroris jakut didisposal , warga tepuk tangan',
 'a': ('!',),
 'i': (6,),
 's': 'duarr ! bahan peledak terduga teroris jakut didisposal , warga tepuk tangan'}

# Feature Extraction: Generate, Dictionary Preprocess [100%]

In [ ]:
# Resources: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Had to modify & adapt most of the code in the tutorial since this isn't translation & data preprocessing is different

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# count the overlap; we need function for this;
def get_overlap(a, b):
    a_counter = collections.Counter(a.split())
    b_counter = collections.Counter(b.split())
    overlap = a_counter & b_counter
    return overlap

def get_total_overlap(a, b):
    return len(list(get_overlap(a, b).elements()))

In [ ]:

noise_chance = 0.4
param_t = 4

def prepareData():
    input_lang = Lang("input")
    output_lang = Lang("output")

    d_pos_a = []
    pairs_pos = []

    for sentence in nonclickbait_l_c:
        c = get_c(sentence, 1, param_t)
        a = get_a(sentence, 1, param_t)
        d_pos_a.append(a)
        pairs_pos.append([c, a, sentence])

    d_neg_a = []
    pairs_neg = []

    for sentence in clickbait_l_c:
        c = get_c(sentence, 0, param_t)
        a = get_a(sentence, 0, param_t)
        d_neg_a.append(a)
        pairs_neg.append([c, a, sentence])

    #adding noise for pos
    for pair in pairs_pos:
        if random.random() < noise_chance:
            real_a = pair[1].split()

            if(len(real_a) == 0):
                continue

            for a in d_pos_a:
                if(len(a) == 0):
                    continue
                a = a.split()
                overlap = get_total_overlap(' '.join(real_a), ' '.join(a))
                if overlap > 0 and ((overlap == len(real_a) - 1 and len(real_a) - len(a) == 1) or (overlap == len(real_a) and len(real_a) - len(a) == -1)):
                    real_a = a
                    break;
            pair[1] = ' '.join(real_a)

    #adding noise for neg
    for pair in pairs_neg:
        if random.random() < noise_chance:
            real_a = pair[1].split()
            if(len(real_a) == 0):
                continue

            for a in d_neg_a:
                if(len(a) == 0):
                    continue
                a = a.split()

                overlap = get_total_overlap(' '.join(real_a), ' '.join(a))
                if overlap > 0 and ((overlap == len(real_a) - 1 and len(real_a) - len(a) == 1) or (overlap == len(real_a) and len(real_a) - len(a) == -1)):
                    real_a = a
                    break;
            pair[1] = ' '.join(real_a)

    pairs = np.concatenate((pairs_pos, pairs_neg), 0)
    for pair in pairs:
        input_lang.addSentence(pair[0])
        input_lang.addSentence(pair[1])
        output_lang.addSentence(pair[2])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs, pairs_neg, pairs_pos

input_lang, output_lang, pairs, pairs_click, pairs_nonclick = prepareData()

pairs_click

/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_45599/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)


Counted words:
input 18304
output 18268


[[' driver di bekasi antar pesanan makanan pakai sepeda',
  'viral ! ojol',
  'viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda'],
 ['ada motor nyangkut bambu di sleman , ',
  'di atas kok bisa ?',
  'ada motor nyangkut di atas bambu di sleman , kok bisa ?'],
 ['pesan gamblang poyuono menolak revisi uu kpk',
  '',
  'pesan gamblang poyuono menolak revisi uu kpk'],
 [' lima kendaraan alami kecelakaan beruntun di cipularang',
  'detik-detik',
  'detik-detik lima kendaraan alami kecelakaan beruntun di cipularang'],
 [' maling di rumah mewah jakut terekam cctv bingung cari jalan kabur',
  'kocak !',
  'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur'],
 ['hampir seluas tantangan sintang siapkan generasi emas',
  'jabar ,',
  'hampir seluas jabar , ini tantangan sintang siapkan generasi emas'],
 [' diduga baku tembak di sleman , polisi',
  'viral video ini kata',
  'viral video diduga baku tembak di sleman , ini kata polisi'],
 ['waspada ada penipua

In [ ]:
pc = pd.DataFrame(pairs_click)
pc

,0,1,2
0,driver di bekasi antar pesanan makanan pakai ...,viral ! ojol,viral ! driver ojol di bekasi antar pesanan ma...
1,"ada motor nyangkut bambu di sleman ,",di atas kok bisa ?,"ada motor nyangkut di atas bambu di sleman , k..."
2,pesan gamblang poyuono menolak revisi uu kpk,,pesan gamblang poyuono menolak revisi uu kpk
3,lima kendaraan alami kecelakaan beruntun di c...,detik-detik,detik-detik lima kendaraan alami kecelakaan be...
4,maling di rumah mewah jakut terekam cctv bing...,kocak !,kocak ! maling di rumah mewah jakut terekam cc...
...,...,...,...
6285,ungkap perasaan karena gelandangan bakal dike...,tamara bleszynski miris dipuji,tamara bleszynski ungkap perasaan miris karena...
6286,istana bantah jokowi tak peka pada korban kabu...,' bareng,istana bantah jokowi tak peka pada korban kabu...
6287,"rilis special ' jelang , konsep dan koreograf...",twice teaser mv ' feel comeback,twice rilis teaser mv ' feel special ' jelang ...
6288,ada niat tambah momongan tanpa ikut program ha...,,ada niat tambah momongan tanpa ikut program ha...


In [ ]:
pnc = pd.DataFrame(pairs_nonclick)
pnc

,0,1,2
0,"masuk radar pilwalkot medan , menantu jokowi b...",,"masuk radar pilwalkot medan , menantu jokowi b..."
1,malaysia sudutkan ri : isu kabut asap hingga i...,,malaysia sudutkan ri : isu kabut asap hingga i...
2,"kemensos salurkan rp 7 , 3 m bagi korban kerus...",,"kemensos salurkan rp 7 , 3 m bagi korban kerus..."
3,mpr : amandemen uud 1945 tak akan melebar ke m...,,mpr : amandemen uud 1945 tak akan melebar ke m...
4,"peringati tahun baru islam , banyuwangi kembal...",,"peringati tahun baru islam , banyuwangi kembal..."
...,...,...,...
5292,"wamena papua kembali membara , kantor bupati d...",,"wamena papua kembali membara , kantor bupati d..."
5293,bmkg angkat bicara soal langit merah jambi,,bmkg angkat bicara soal langit merah jambi
5294,operasional bandara wamena dihentikan akibat r...,,operasional bandara wamena dihentikan akibat r...
5295,"asap karhutla riau mulai merambah ke nias , bm...",,"asap karhutla riau mulai merambah ke nias , bm..."


In [ ]:
# for pair in pairs:
#     input_lang.addSentence(pair[0])
#     output_lang.addSentence(pair[1])
# for pair in pairs:
#     input_lang.addSentence(pair[1])
#     output_lang.addSentence(pair[0])

print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)

Counted words:
input 13924
output 13896


# Dataset:  Train, Validation, Test Split [100% Done]

In [ ]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(pairs, test_size=0.3, random_state=42)
# train_data, test_data = train_test_split(pairs, test_size=0.3, random_state=42)
train_data, test_data  = train_test_split(pairs, test_size=0.2, random_state=42)
train_data, validation_data = train_test_split(train_data, test_size=0.18, random_state=42)

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")
print(f"Number of validation examples: {len(validation_data)}")

Number of training examples: 9840
Number of testing examples: 3000
Number of validation examples: 2160


In [ ]:
print(f"{train_data[0]}")
print(test_data[0])
print(validation_data[0])

['kesaksian petugas bpdb yang bertemu beruang buas saat jalankan tugas padamkan kebakaran hutan', 'kesaksian petugas bpdb yang bertemu beruang buas saat jalankan tugas padamkan kebakaran hutan', 1]
['bekasi gandeng 3 perusahaan olah sampah jadi listrik', 'bekasi gandeng 3 perusahaan olah sampah jadi listrik', 0]
['menjajal lenovo v330-14arr , laptop 14 inci dengan amd ryzen 3', 'menjajal lenovo v330-14arr , laptop 14 inci dengan amd ryzen 3', 1]



# STAGE 3: NN Model : Encoder, Decoder, Seq2Seq [100% Done]

In [ ]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, word_vec_size, hidden_size, weight):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(input_size, word_vec_size)
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, word_vec_size, output_size, weight):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(output_size, word_vec_size)
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.maxout = Maxout(1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.maxout(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# STAGE 5: Improvement : Add fast text model

In [ ]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gensim.models.fasttext import load_facebook_model

# ft = load_facebook_model('fastext_model_tweet.bin')

In [ ]:
# fast_text_path = "2022_thesis_styletransfer/cp_fastext_model_tweet.bin"
# ft = load_facebook_model(fast_text_path)


fast_text_path = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/cc.id.300.bin"
ft = load_facebook_model(fast_text_path)

# fast_text_path_mega = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/fasttext.4B.id.300.epoch5.uncased.bin"
# ft = load_facebook_model(fast_text_path_mega)

In [ ]:
ft.vector_size

300

In [ ]:
embedding_matrix_ft_input = np.random.random((input_lang.n_words , ft.vector_size))
embedding_matrix_ft_output = np.random.random((output_lang.n_words, ft.vector_size))

embedding_matrix_ft_input_test = np.random.random((input_lang.n_words , ft.vector_size))

# torch.zeros(max_length, encoder_a.hidden_size, device=device)

In [ ]:
print(input_lang.n_words)
print(ft.vector_size)

17836
300


In [ ]:
for i, word in input_lang.index2word.items():
    try:
        embedding_matrix_ft_input[i] = ft.wv[word]
    except:
        pass

for i, word in output_lang.index2word.items():
    try:
        embedding_matrix_ft_output[i] = ft.wv[word]
    except:
        pass

In [ ]:
embedding_matrix_ft_input = torch.from_numpy(embedding_matrix_ft_input)
embedding_matrix_ft_output = torch.from_numpy(embedding_matrix_ft_output)

In [ ]:
embedding_matrix_ft_output = embedding_matrix_ft_output.float()
embedding_matrix_ft_input = embedding_matrix_ft_input.float()

print(embedding_matrix_ft_output)

tensor([[ 0.0572,  0.2053, -0.0637,  ...,  0.0979,  0.0835, -0.0297],
        [-0.1267, -0.0145,  0.1540,  ..., -0.0899,  0.0128, -0.2284],
        [ 0.0092,  0.0295, -0.0068,  ..., -0.0069, -0.0752, -0.0411],
        ...,
        [-0.0043, -0.0099,  0.0077,  ...,  0.0365, -0.0052,  0.0409],
        [ 0.0327, -0.0176,  0.0261,  ...,  0.0289, -0.0305,  0.0290],
        [-0.0059, -0.0435, -0.0289,  ...,  0.0608,  0.0280, -0.0045]])


In [ ]:
# torch.Size([18268, 300])
embedding_matrix_ft_output.size()

torch.Size([18268, 300])

In [ ]:
# torch.Size([17836, 300])
embedding_matrix_ft_input.size()

torch.Size([17836, 300])

# STAGE 3: Word to Index Conversion

In [ ]:
#Preparing Training Data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # print("Indexes", indexes )
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_c_tensor = tensorFromSentence(input_lang, pair[0])
    input_a_tensor = tensorFromSentence(input_lang, pair[1])
    target_tensor = tensorFromSentence(output_lang, pair[2])
    # print(input_c_tensor)
    # print(input_a_tensor)
    return (input_c_tensor, input_a_tensor, target_tensor)

In [ ]:
training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(1)]
training_pairs

[(tensor([[  614],
          [ 7197],
          [ 1613],
          [   34],
          [  408],
          [    6],
          [ 1289],
          [ 2056],
          [   17],
          [  249],
          [10825],
          [    1]]),
  tensor([[13],
          [ 1]]),
  tensor([[  613],
          [ 7195],
          [ 1607],
          [   33],
          [  407],
          [    6],
          [ 1282],
          [ 2050],
          [   16],
          [  248],
          [10826],
          [    1]]))]

In [ ]:
#Training
MAX_LENGTH = 50

teacher_forcing_ratio = 0.5


def train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a, decoder, encoder_c_optimizer, encoder_a_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_c_hidden = encoder_c.initHidden()
    encoder_a_hidden = encoder_a.initHidden()

    # print("encoder_a_hidden", encoder_a_hidden)

    encoder_a_optimizer.zero_grad()
    encoder_c_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_c_length = input_c_tensor.size(0)
    input_a_length = input_a_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
    encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

    loss = 0

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(
            input_c_tensor[ei], encoder_c_hidden)
        encoder_c_outputs[ei] = encoder_c_output[0, 0]

    for ei in range(input_a_length):
        encoder_a_output, encoder_a_hidden = encoder_a(
            input_a_tensor[ei], encoder_a_hidden)
        encoder_a_outputs[ei] = encoder_a_output[0, 0]


    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_c_optimizer.step()
    encoder_a_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

# STAGE 3: NN Model : Training Architecture

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(pairs, test_size=0.1, random_state=42)
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")
print(f"{train_data[0]}")
print(test_data[0])

Number of training examples: 13500
Number of testing examples: 1500
['satu calon kades kalap dan histeris usai dibatalkan mendadak', 'satu calon kades kalap dan histeris usai dibatalkan mendadak', 0]
['bekasi gandeng 3 perusahaan olah sampah jadi listrik', 'bekasi gandeng 3 perusahaan olah sampah jadi listrik', 0]


In [ ]:
def trainIters(
        encoder_a,
        encoder_c,
        decoder,
        n_iters,

        criterion,
        encoder_c_optimizer,
        encoder_a_optimizer,
        decoder_optimizer,

        print_every=1000,
        plot_every=100,
        learning_rate=0.01):

    encoder_a.train()
    encoder_c.train()
    decoder.train()

    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
    # encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
    # decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    # training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(n_iters)]
    training_pairs = [tensorsFromPair(train_data[i])for i in range(n_iters)]
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    # criterion = nn.NLLLoss()

    ouput_loss_total = 0

    for iter in range(1, n_iters + 1):

        training_pair = training_pairs[iter - 1]
        input_a_tensor = training_pair[0]
        input_c_tensor = training_pair[1]
        target_tensor = training_pair[2]

        loss = train(input_c_tensor, input_a_tensor, target_tensor,
                     encoder_c, encoder_a, decoder,
                     encoder_c_optimizer, encoder_c_optimizer, decoder_optimizer,
                     criterion)

        print_loss_total += loss
        plot_loss_total += loss
        ouput_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
    #
    #     if iter % plot_every == 0:
    #         plot_loss_avg = plot_loss_total / plot_every
    #         plot_losses.append(plot_loss_avg)
    #         plot_loss_total = 0
    #
    # showPlot(plot_losses)

    return ouput_loss_total/n_iters

In [ ]:
def evaluating(
        encoder_a,
        encoder_c,
        decoder,

        criterion,
        n_iters,
        plot_every : int =10,
        max_length=MAX_LENGTH
):

    encoder_a.eval()
    encoder_c.eval()
    decoder.eval()

    plot_losses = []
    plot_loss_total = 0
    loss_total_output = 0

    # training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(n_iters)]
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    training_pairs = [tensorsFromPair(test_data[i])for i in range(n_iters)]

    start: float = time.time()

    with torch.no_grad():
        for iter  in  range(1, n_iters + 1):
            # taking the pair and extract the c, a, target
            training_pair = training_pairs[iter - 1]
            input_c_tensor = training_pair[0]
            input_a_tensor = training_pair[1]
            target_tensor = training_pair[2]

            # START: eval func
            # init hidden
            encoder_c_hidden = encoder_c.initHidden()
            encoder_a_hidden = encoder_a.initHidden()

            # len of sentence of c, a, t, that already; e.g input_c_length= len(input_c_tensor)
            input_c_length = input_c_tensor.size(0)
            input_a_length = input_a_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
            encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

            loss = 0

            for ei in range(input_c_length):
                encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei], encoder_c_hidden)
                encoder_c_outputs[ei] = encoder_c_output[0, 0]

            for ei in range(input_a_length):
                encoder_a_output, encoder_a_hidden = encoder_a(input_a_tensor[ei], encoder_a_hidden)
                encoder_a_outputs[ei] = encoder_a_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            if use_teacher_forcing:
                # Teacher forcing: Feed the target as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    loss += criterion(decoder_output, target_tensor[di])
                    decoder_input = target_tensor[di]  # Teacher forcing

            else:
            # Without teacher forcing: use its own predictions as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += criterion(decoder_output, target_tensor[di])
                    if decoder_input.item() == EOS_token:
                        break

            l = loss.item() / target_length
            # END: Eval Func

            loss_total_output += l
            plot_loss_total += l

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                # print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                #                              iter, iter / n_iters * 100, plot_loss_avg))
                plot_loss_total = 0

        # showPlot(plot_losses)

    return loss_total_output/ n_iters

In [ ]:
# word_vec_size = 128
word_vec_size = ft.vector_size
hidden_size = 512

# encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words).to(device)

encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)

decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)

In [ ]:
# trainIters(encoder_c, encoder_a, decoder, 200, print_every=100)
# len(pairs_click)
print(len(pairs) - len(pairs_click))
print(len(pairs_click))
print(len(pairs))

5297
3316
8613


In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
n_t = len(train_data)
n_e = len(test_data)
N_EPOCHS = 20
best_valid_loss = float('inf')

learning_rate=0.1
# learning_rate = 0.001

encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)

# encoder_optimizer = optim.Adam(encoder1.parameters(),lr=learning_rate)
# decoder_optimizer = optim.Adam(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=learning_rate)

# PATH = "delete_only_model.pt"
PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_retrieve/delete_retrive_model.pt"
MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_retrieve/"

TRAINING_DATASET_SIZE = n_t
EVALUATION_DATASET_SIZE = n_e

# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

plot_train_loss = []
plot_eval_loss = []

perplexity_train = []
perplexity_eval = []


for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = trainIters(
        encoder_a,
        encoder_c,
        decoder,
        TRAINING_DATASET_SIZE,

        criterion,
        encoder_c_optimizer,
        encoder_a_optimizer,
        decoder_optimizer,
        print_every=50
    )

    valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=50)

    plot_train_loss.append(train_loss)
    plot_eval_loss.append(valid_loss)


    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder_a.state_dict(), MAIN_PATH +  'encoder_a-model.pt')
        torch.save(encoder_c.state_dict(), MAIN_PATH  + 'encoder_c-model.pt')
        torch.save(decoder.state_dict(), MAIN_PATH + 'decoder-model.pt')

        torch.save({
            'epoch': epoch,
            'encoder_c_state_dict': encoder_c.state_dict(),
            'encoder_a_state_dict': encoder_a.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'encoder_c_optimizer_state_dict': encoder_c_optimizer.state_dict(),
            'encoder_a_optimizer_state_dict': encoder_a_optimizer.state_dict(),
            'decoder_optimizer_state_dict': decoder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")

showPlot(plot_train_loss)
showPlot(plot_eval_loss)

0m 35s (- 22m 58s) (50 2%) 8.7313
0m 57s (- 18m 12s) (100 5%) 5.5482
1m 19s (- 16m 25s) (150 7%) 5.6318
1m 46s (- 15m 54s) (200 10%) 5.9161
2m 8s (- 14m 56s) (250 12%) 4.7780
2m 30s (- 14m 14s) (300 15%) 5.3833
2m 52s (- 13m 33s) (350 17%) 5.5071
3m 16s (- 13m 4s) (400 20%) 5.9503
3m 36s (- 12m 26s) (450 22%) 4.7532
3m 59s (- 11m 57s) (500 25%) 5.2679
4m 18s (- 11m 22s) (550 27%) 4.6226
4m 39s (- 10m 53s) (600 30%) 4.7787
4m 58s (- 10m 19s) (650 32%) 4.2902
5m 17s (- 9m 50s) (700 35%) 4.2425
5m 40s (- 9m 28s) (750 37%) 4.7355
6m 0s (- 9m 0s) (800 40%) 3.5813
6m 27s (- 8m 44s) (850 42%) 5.8347
6m 49s (- 8m 20s) (900 45%) 3.9243
7m 13s (- 7m 59s) (950 47%) 4.8169
7m 37s (- 7m 37s) (1000 50%) 4.5308
8m 0s (- 7m 14s) (1050 52%) 4.6374
8m 18s (- 6m 47s) (1100 55%) 3.2666
8m 37s (- 6m 22s) (1150 57%) 3.7249
9m 2s (- 6m 1s) (1200 60%) 4.7659
9m 23s (- 5m 37s) (1250 62%) 4.4472
9m 47s (- 5m 16s) (1300 65%) 3.9644
10m 13s (- 4m 55s) (1350 67%) 4.6762
10m 37s (- 4m 33s) (1400 70%) 4.0658
10m 57s

# Stage 3: Testing the DeleteRetrieve to Generate Text

In [ ]:
def encoderStep(encoder_c, encoder_a, c, a, max_length=MAX_LENGTH):
    input_c_tensor = tensorFromSentence(input_lang, c)
    input_a_tensor = tensorFromSentence(input_lang, a)

    input_c_length = input_c_tensor.size()[0]
    input_a_length = input_a_tensor.size()[0]

    encoder_c_hidden = encoder_c.initHidden()
    encoder_a_hidden = encoder_a.initHidden()

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
    encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei],
                                                       encoder_c_hidden)
        encoder_c_outputs[ei] += encoder_c_output[0, 0]


    for ei in range(input_a_length):
        encoder_a_output, encoder_a_hidden = encoder_a(input_a_tensor[ei],
                                                       encoder_a_hidden)
        encoder_a_outputs[ei] += encoder_a_output[0, 0]



    return torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

In [ ]:
def get_c_embedding(encoder_c, c, max_length=MAX_LENGTH):
    input_c_tensor = tensorFromSentence(input_lang, c)

    input_c_length = input_c_tensor.size()[0]

    encoder_c_hidden = encoder_c.initHidden()

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei],
                                                       encoder_c_hidden)
        encoder_c_outputs[ei] += encoder_c_output[0, 0]

    return encoder_c_hidden

In [ ]:
ALPHA = 0.7

class Node:
    def __init__(self, p_t, i, d_hidden, lvl, node_parent=None):
        self.p_t = p_t
        self.i = i
        self.d_hidden = d_hidden
        self.lvl = lvl
        self.node_parent = node_parent

        self.p_sentence = self.sentenceProb()

    def prepareToDecode(self):
        d_in = self.i.squeeze()#.detatch()
        return d_in, self.d_hidden

    def sentenceProb(self):
        if self.node_parent == None:
            return self.p_t
        return self.node_parent.sentenceProb() + self.p_t #addition because of log

    def normProb(self):
        #Normalised probability
        #         return 1/((self.lvl+1)**ALPHA) * self.sentenceProb()
        #perplexity
        return torch.exp(self.sentenceProb()) ** (-1/(self.lvl+1))

    def getToken(self):
        return output_lang.index2word[self.i.item()]

    def getTokens(self):
        token = [self.getToken()]

        if(self.node_parent == None):
            return token
        else:
            return self.node_parent.getTokens() + token

    def getSentence(self):
        return ' '.join(self.getTokens())

In [ ]:
BEAM_WIDTH = 10

def evaluate(encoder_c, encoder_a, decoder, c, a, max_length=MAX_LENGTH):
    with torch.no_grad():
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoderStep(encoder_c, encoder_a, c, a)
        decoded_words = []

        #BEAM SEARCH

        nodes = []
        finished_nodes = []
        for di in range(max_length):
            if di == 0:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                top_ps, top_is = decoder_output.data.topk(BEAM_WIDTH)
                top_ps = top_ps.view(-1)
                top_is = top_is.view(-1)
                for index in range(len(top_is)):
                    p_t = top_ps[index]
                    i = top_is[index]
                    nodes.append(Node(p_t, i, decoder_hidden, di))
            else:
                prev_nodes = [x for x in nodes if x.lvl == di-1]
                prev_nodes = sorted(prev_nodes, key=lambda x: x.p_sentence.item(), reverse=True)
                prev_nodes = prev_nodes[:BEAM_WIDTH]
                nodes = []
                for node in prev_nodes:
                    decoder_output, decoder_hidden = decoder(*node.prepareToDecode())
                    top_ps, top_is = decoder_output.data.topk(BEAM_WIDTH)
                    top_ps = top_ps.view(-1)
                    top_is = top_is.view(-1)

                    for index in range(len(top_is)):
                        p_t = top_ps[index]
                        i = top_is[index]
                        child_node = Node(p_t, i, decoder_hidden, di, node)

                        if i.item() == EOS_token:
                            finished_nodes.append(child_node)
                        else:
                            nodes.append(child_node)

        final_node = sorted(finished_nodes, key=lambda x: x.p_sentence.item(), reverse=True)[0]
        return final_node.getTokens()

# evaluate(encoder_c, encoder_a, decoder, "the food", "great")

In [ ]:
def evaluateWithoutBeamSearch(encoder_c, encoder_a, decoder, c, a, max_length=MAX_LENGTH):
    with torch.no_grad():
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoderStep(encoder_c, encoder_a, c, a)
        decoded_words = []

        #BEAM SEARCH
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            topv, topi = F.softmax(decoder_output).data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
# ' '.join(evaluateWithoutBeamSearch(encoder_c, encoder_a, decoder, "the food is", "great"))

In [ ]:
def evaluateRandomly(encoder_c, encoder_a, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder_c, encoder_a, decoder, pair[0], pair[1])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
evaluateRandomly(encoder_c, encoder_a, decoder)

> agus rahardjo pastikan aktivitas kpk berjalan seperti biasa
= 
< , EOS

> haris azhar kritik menkumham soal gel ngan didenda rp 1 juta : gembel enggak nonton pidato 
= anda
< , EOS

> ishlah , kunci pengurusan anak yatim menurut alquran
= apa kata
< , EOS

>  mirip , polisi tilang ayah kandungnya
= ini ternyata
< , EOS

> anies cabut kasasi perkara lahan sodetan ciliwung era ahok di ma
= 
< , EOS

> tinggal di lingkungan sineas , tetangga joko anwar ada tara basro sampai dimas anggara
= 
< , EOS

> 850 personel polisi siap amankan itdbi 2019
= 
< , EOS

> menu sehat dari sdn sebabkan balita meninggal di koja 
= ?
< , EOS

>  wartawan rekam tindakan represif aparat polisi jadi 
= viral , sikap ini sorotan
< , EOS

> tiga bulan , 400 tangki air dikirim ke warga lamongan atasi kekeringan
= 
< , EOS



In [ ]:
def DeleteAndRetrieve(sentence, style):
    c_src = separate(sentence, 0)['c']
    a_tgt = ' '.join(retrieve(sentence, not style)['a'])
    return ' '.join(evaluate(encoder_c, encoder_a, decoder, c_src, a_tgt))

# STAGE 4: ADDITIONAL: FULL COPY ONLY ONE EPOCH

In [ ]:
# class Maxout(nn.Module):
#     def __init__(self, pool_size):
#         super().__init__()
#         self._pool_size = pool_size
#
#     def forward(self, x):
#         assert x.shape[1] % self._pool_size == 0, \
#             'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
#         m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
#         return m

In [ ]:
# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, word_vec_size, hidden_size):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         self.embedding = nn.Embedding(input_size, word_vec_size)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#
#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding(output_size, word_vec_size)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#
#     def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
#         output = self.maxout(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
# #Preparing Training Data
# def indexesFromSentence(lang, sentence):
#     return [lang.word2index[word] for word in sentence.split(' ')]
#
#
# def tensorFromSentence(lang, sentence):
#     indexes = indexesFromSentence(lang, sentence)
#     indexes.append(EOS_token)
#     return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)
#
#
# def tensorsFromPair(pair):
#     input_c_tensor = tensorFromSentence(input_lang, pair[0])
#     input_a_tensor = tensorFromSentence(input_lang, pair[1])
#     target_tensor = tensorFromSentence(output_lang, pair[2])
#     return (input_c_tensor, input_a_tensor, target_tensor)

In [ ]:
# #Training
# MAX_LENGTH = 50
#
# teacher_forcing_ratio = 0.5
#
#
# def train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a, decoder, encoder_c_optimizer, encoder_a_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
#     encoder_c_hidden = encoder_c.initHidden()
#     encoder_a_hidden = encoder_a.initHidden()
#
#     encoder_a_optimizer.zero_grad()
#     encoder_c_optimizer.zero_grad()
#
#     decoder_optimizer.zero_grad()
#
#     input_c_length = input_c_tensor.size(0)
#     input_a_length = input_a_tensor.size(0)
#
#     target_length = target_tensor.size(0)
#
#     encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
#     encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)
#
#     loss = 0
#
#     for ei in range(input_c_length):
#         encoder_c_output, encoder_c_hidden = encoder_c(
#             input_c_tensor[ei], encoder_c_hidden)
#         encoder_c_outputs[ei] = encoder_c_output[0, 0]
#
#     for ei in range(input_a_length):
#         encoder_a_output, encoder_a_hidden = encoder_a(
#             input_a_tensor[ei], encoder_a_hidden)
#         encoder_a_outputs[ei] = encoder_a_output[0, 0]
#
#
#     decoder_input = torch.tensor([[SOS_token]], device=device)
#
#     decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)
#
#     use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
#
#     if use_teacher_forcing:
#         # Teacher forcing: Feed the target as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(
#                 decoder_input, decoder_hidden)
#             loss += criterion(decoder_output, target_tensor[di])
#             decoder_input = target_tensor[di]  # Teacher forcing
#
#     else:
#         # Without teacher forcing: use its own predictions as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(
#                 decoder_input, decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach()  # detach from history as input
#
#             loss += criterion(decoder_output, target_tensor[di])
#             if decoder_input.item() == EOS_token:
#                 break
#
#     loss.backward()
#
#     encoder_c_optimizer.step()
#     encoder_a_optimizer.step()
#     decoder_optimizer.step()
#
#     return loss.item() / target_length

In [ ]:
# import time
# import math
#
# def asMinutes(s):
#     m = math.floor(s / 60)
#     s -= m * 60
#     return '%dm %ds' % (m, s)
#
#
# def timeSince(since, percent):
#     now = time.time()
#     s = now - since
#     es = s / (percent)
#     rs = es - s
#     return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
# import matplotlib.pyplot as plt
# plt.switch_backend('agg')
# import matplotlib.ticker as ticker
# import numpy as np
#
#
# def showPlot(points):
#     plt.figure()
#     fig, ax = plt.subplots()
#     # this locator puts ticks at regular intervals
#     loc = ticker.MultipleLocator(base=0.2)
#     ax.yaxis.set_major_locator(loc)
#     plt.plot(points)

In [ ]:
# def trainIters(encoder_a, encoder_c, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
#     start = time.time()
#     plot_losses = []
#     print_loss_total = 0  # Reset every print_every
#     plot_loss_total = 0  # Reset every plot_every
#
#     encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
#     encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)
#     training_pairs = [tensorsFromPair(random.choice(pairs))
#                       for i in range(n_iters)]
#     criterion = nn.NLLLoss()
#
#     for iter in range(1, n_iters + 1):
#         training_pair = training_pairs[iter - 1]
#         input_a_tensor = training_pair[0]
#         input_c_tensor = training_pair[1]
#
#         target_tensor = training_pair[2]
#         loss = train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a,
#                      decoder, encoder_c_optimizer, encoder_c_optimizer, decoder_optimizer, criterion)
#         print_loss_total += loss
#         plot_loss_total += loss
#
#         if iter % print_every == 0:
#             print_loss_avg = print_loss_total / print_every
#             print_loss_total = 0
#             print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
#                                          iter, iter / n_iters * 100, print_loss_avg))
#
#         if iter % plot_every == 0:
#             plot_loss_avg = plot_loss_total / plot_every
#             plot_losses.append(plot_loss_avg)
#             plot_loss_total = 0
#
#     showPlot(plot_losses)

In [ ]:
# word_vec_size = 128
# hidden_size = 512
# encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
#
# decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words).to(device)

In [ ]:
# trainIters(encoder_c, encoder_a, decoder, 8613, print_every=200)

# STAGE 5: DELETE ONLY FUNCTION

In [ ]:
# Resources: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Had to modify & adapt most of the code in the tutorial since this isn't translation & data preprocessing is different

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
param_t = 4

def prepareData():
    input_lang = Lang("input")
    output_lang = Lang("output")

    pairs = []
    pairs_pos = []
    pairs_neg = []

    for sentence in clickbait_l_c:
        comp = [get_c(sentence, 1, param_t), sentence, 1]
        pairs_neg.append(comp)
        pairs.append(comp)

    for sentence in nonclickbait_l_c:
        comp = [get_c(sentence, 0, param_t), sentence, 0]
        pairs_pos.append(comp)
        pairs.append(comp)

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs, pairs_pos, pairs_neg

In [ ]:
# input_lang, output_lang, pairs, pairs_nonclick, pairs_click  = prepareData()
_ , _ , pairs, pairs_nonclick, pairs_click  = prepareData()

<ipython-input-21-3f72f6d89c87>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)
<ipython-input-21-3f72f6d89c87>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)
<ipython-input-21-3f72f6d89c87>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ng

Counted words:
input 17839
output 18268


In [ ]:
# for pair in pairs:
#     input_lang.addSentence(pair[0])
#     output_lang.addSentence(pair[1])
# for pair in pairs:
#     input_lang.addSentence(pair[1])
#     output_lang.addSentence(pair[0])

print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)

Counted words:
input 17836
output 18268


In [ ]:
# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, word_vec_size, hidden_size, weight, dropout):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         # self.embedding = nn.Embedding(input_size, word_vec_size)
#         self.embedding = nn.Embedding.from_pretrained(weight)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.dropout = nn.Dropout(dropout)
#
#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         embedded = self.dropout(embedded)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input, ENC_DROPOUT).to(device)


class EncoderRNN(nn.Module):
    def __init__(self, input_size, word_vec_size, hidden_size, weight):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding.from_pretrained(weight)
        self.gru = nn.GRU(word_vec_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, word_vec_size, hidden_size, weight):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.layer = 8
#
#         # self.embedding = nn.Embedding(input_size, word_vec_size)
#         self.embedding = nn.Embedding.from_pretrained(weight)
#         self.gru = nn.GRU(word_vec_size, hidden_size, self.layer)
#
#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class StyleEmbedder(nn.Module):
    def __init__(self, num_styles, dimensions):
        super(StyleEmbedder, self).__init__()
        self.dimensions = dimensions
        self.embedding = nn.Embedding(num_styles, dimensions)

    def forward(self, input):
        embedded = self.embedding(input).view(1, 1, -1)
        return embedded

In [ ]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [ ]:


# Dec 7, chat gpt
# In this version, we have changed the GRU cell to an LSTM cell and added an additional layer. We have also added a dropout regularization to prevent overfitting. Finally, we have removed the Maxout activation function and replaced it with a LogSoftmax function, which is more commonly used for output activation in natural language processing tasks.
# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight, dropout=0.2):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding.from_pretrained(weight)
#
#         self.gru = nn.LSTM(word_vec_size, hidden_size, dropout=dropout)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#
#     def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)



# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, word_vec_size, hidden_size, weight):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding.from_pretrained(weight)
#         self.gru = nn.GRU(word_vec_size, hidden_size)

#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden

#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)


# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding.from_pretrained(weight)

#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)

# def forward(self, input, hidden):
#     output = self.embedding(input).view(1, 1, -1)
#     output = self.maxout(output)
#     output, hidden = self.gru(output, hidden)
#     output = self.softmax(self.out(output[0]))
#     return output, hidden

# def initHidden(self):
#     return torch.zeros(1, 1, self.hidden_size, device=device)


# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, word_vec_size, hidden_size, weight, dropout=0.2):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         self.embedding = nn.Embedding.from_pretrained(weight)
#         self.gru = nn.LSTM(word_vec_size, hidden_size, dropout=dropout)
#
#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)


# class StyleEmbedder(nn.Module):
#     def __init__(self, num_styles, dimensions, dropout=0.2):
#         super(StyleEmbedder, self).__init__()
#         self.dimensions = dimensions
#         self.embedding = nn.Embedding(num_styles, dimensions)
#         self.dropout = nn.Dropout(p=dropout)
#         self.relu = nn.ReLU()
#
#     def forward(self, input):
#         embedded = self.embedding(input).view(1, 1, -1)
#         embedded = self.dropout(embedded)
#         embedded = self.relu(embedded)
#         return embedded



In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, word_vec_size, output_size, weight):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.maxout = Maxout(1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.maxout(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight, dropout):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         # self.layer = 8;
#
#         # self.embedding = nn.Embedding(output_size, word_vec_size)
#         self.embedding = nn.Embedding.from_pretrained(weight)
#
#         # self.gru = nn.GRU(word_vec_size, hidden_size, self.layer)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#         self.dropout = nn.Dropout(dropout)
#
#     def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
#         output = self.maxout(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)


# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         # self.embedding = nn.Embedding(output_size, word_vec_size)
#         self.embedding = nn.Embedding.from_pretrained(weight)
#
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#         self.attention = nn.Linear(hidden_size + word_vec_size, hidden_size)
#
#
#     def forward(self, input, hidden):
#         # Calculate the attention weights
#
#         print("input type", type(input) )
#         print("hidden type", type(hidden) )
#
#         print("input", input.size())
#         print("hidden", hidden.size())
#
#         test2 = (input.view(1, 1, -1), hidden)
#
#         print("input_view", len(test2))
#
#         attn_weights = F.softmax(self.attention(torch.cat(test2, 1)))
#         # attn_weights = F.softmax(self.attention(torch.cat((input, hidden), 1)))
#
#         # Create a weighted sum of the input and hidden state using the attention weights as coefficients
#         context = attn_weights * input + (1 - attn_weights) * hidden
#
#         # Pass the weighted sum through the GRU layer along with the embedded input to produce the decoder's output
#         output, hidden = self.gru(torch.cat((context, self.embedding(input).view(1, 1, -1)), 2), hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)



#
# MAX_LENGTH = 20
# class AttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size, weight, dropout_p=0.1, max_length=MAX_LENGTH):
#         super(AttnDecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.dropout_p = dropout_p
#         self.max_length = max_length
#
#         # hidden_size = word_vec_size;
#         # self.embedding = nn.Embedding.from_pretrained(weight)
#         self.embedding = nn.Embedding(self.output_size, self.hidden_size)
#         self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
#         self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
#         self.dropout = nn.Dropout(self.dropout_p)
#         self.gru = nn.GRU(self.hidden_size, self.hidden_size)
#         self.out = nn.Linear(self.hidden_size, self.output_size)
#
#     def forward(self, input, hidden, encoder_outputs):
#         embedded = self.embedding(input).view(1, 1, -1)
#         embedded = self.dropout(embedded)
#
#         attn_weights = F.softmax(
#             self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
#         attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
#
#         output = torch.cat((embedded[0], attn_applied[0]), 1)
#         output = self.attn_combine(output).unsqueeze(0)
#
#         output = F.relu(output)
#         output, hidden = self.gru(output, hidden)
#
#         output = F.log_softmax(self.out(output[0]), dim=1)
#         return output, hidden, attn_weights
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

#
# class Attention(nn.Module):
#     def __init__(self, hidden_size):
#         super(Attention, self).__init__()
#         self.hidden_size = hidden_size
#         self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
#         self.v = nn.Parameter(torch.rand(hidden_size))
#         stdv = 1. / math.sqrt(self.v.size(0))
#         self.v.data.normal_(mean=0, std=stdv)
#
#     def forward(self, hidden, encoder_outputs):
#         timestep = encoder_outputs.size(0)
#         h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
#         encoder_outputs = encoder_outputs.transpose(0, 1)
#         attn_energies = self.score(h, encoder_outputs)
#         return F.softmax(attn_energies, dim=1).unsqueeze(1)
#
#     def score(self, hidden, encoder_outputs):
#
#         print("hidden", hidden)
#         print("encoder_outputs", encoder_outputs)
#
#         energy = F.relu(self.attn(torch.cat([hidden, encoder_outputs], 2)))
#         energy = energy.transpose(1, 2)
#         v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
#         energy = torch.bmm(v, energy)
#         return energy.squeeze(1)
#
#


# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding.from_pretrained(weight)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#         self.attn = Attention(hidden_size)
#
#     def forward(self, input, hidden, encoder_outputs):
#         output = self.embedding(input).view(1, 1, -1)
#         output = self.maxout(output)
#         output, hidden = self.gru(output, hidden)
#         attn_weights = self.attn(output, encoder_outputs)
#         context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
#         output = output.squeeze(0)
#         context = context.squeeze(1)
#         output = self.softmax(self.out(torch.cat([output, context], 1)))
#         return output, hidden, attn_weights
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)


# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         self.embedding = nn.Embedding.from_pretrained(weight)
#
#         # Add an attention layer
#         self.attention = nn.Linear(hidden_size + word_vec_size, hidden_size)
#
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#
#     def forward(self, input, hidden, encoder_outputs):
#         # Calculate the attention weights
#         attn_weights = F.softmax(self.attention(torch.cat((input, hidden), 1)))
#         # attn_weights = F.softmax(self.attention(torch.cat((input.unsqueeze(1), hidden), 1)))
#
#
#         # Create a weighted sum of the encoder outputs using the attention weights as coefficients
#         context = attn_weights.unsqueeze(0).bmm(encoder_outputs.unsqueeze(0))
#
#         # Pass the weighted sum through the GRU layer along with the embedded input to produce the decoder's output
#         output, hidden = self.gru(torch.cat((context, self.embedding(input).view(1, 1, -1)), 2), hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden


In [ ]:
# #Preparing Training Data
# def indexesFromSentence(lang, sentence):
#     return [lang.word2index[word] for word in sentence.split(' ')]


# def tensorFromSentence(lang, sentence):
#     indexes = indexesFromSentence(lang, sentence)
#     indexes.append(EOS_token)
#     return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# def tensorFromStyle(style):
#     #     one_hot_encoded_style = []
#     #     if style:
#     #         one_hot_encoded_style = [1,0]
#     #     else:
#     #         one_hot_encoded_style = [0,1]
#     return torch.tensor(style, dtype=torch.long, device=device).view(-1, 1)

# def tensorsFromPair(pair):
#     input_tensor = tensorFromSentence(input_lang, pair[0])
#     target_tensor = tensorFromSentence(output_lang, pair[1])
#     style_tensor = tensorFromStyle(pair[2])
#     return (input_tensor, style_tensor, target_tensor)  #add style_tensor



#Preparing Training Data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorFromStyle(style):
    #     one_hot_encoded_style = []
    #     if style:
    #         one_hot_encoded_style = [1,0]
    #     else:
    #         one_hot_encoded_style = [0,1]
    return torch.tensor(style, dtype=torch.long, device=device).view(-1, 1)

def checkTensorExist(lang, sentence):
    for word in sentence.split(' '):
        if word not in lang.word2index:
            return False
    return True

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    style_tensor = tensorFromStyle(pair[2])
    return (input_tensor, style_tensor, target_tensor)

In [ ]:
#Trainin

teacher_forcing_ratio = 0.5

MAX_LENGTH = 50


def train(
        input_tensor,
        style_tensor,
        target_tensor,

        encoder,
        style_embedder,
        decoder,

        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,

        criterion,
        max_length=MAX_LENGTH):


    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    style_embedder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    #calculate style embedding
    style_embedding = style_embedder(style_tensor)

    decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2) #TODO: concatenate style embedding

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    style_embedder_optimizer.step()
    decoder_optimizer.step()


    return loss.item() / target_length

In [ ]:
# # chatgpt;
# # MAX_LENGTH = 20
# MAX_LENGTH = 50
# #Trainin
# teacher_forcing_ratio = 0.5
#
# def train(
#         input_tensor,
#         style_tensor,
#         target_tensor,
#
#         encoder,
#         style_embedder,
#         decoder,
#
#         encoder_optimizer,
#         style_embedder_optimizer,
#         decoder_optimizer,
#
#         criterion,
#         max_length=MAX_LENGTH):
#
#
#     encoder_hidden = encoder.initHidden()
#
#     encoder_optimizer.zero_grad()
#     decoder_optimizer.zero_grad()
#     style_embedder_optimizer.zero_grad()
#
#     input_length = input_tensor.size(0)
#     target_length = target_tensor.size(0)
#
#     encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
#
#     loss = 0
#
#     # Encode the input sequence using the encoder
#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[ei], encoder_hidden)
#         encoder_outputs[ei] = encoder_output[0, 0]
#
#     # Compute the embedded style representation
#     style_embedding = style_embedder(style_tensor)
#
#     # Initialize the decoder input and hidden state
#     decoder_input = torch.tensor([[SOS_token]], device=device)
#
#     # Concatenate encoder_hidden and style_embedding along the last dimension
#     decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2)
#     # decoder_hidden = torch.cat((encoder_hidden[0], style_embedding), 2)
#
#
#     # Use teacher forcing if the random value is less than the teacher forcing ratio
#     use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
#
#     # Decode the input sequence using the decoder and compute the loss
#     if use_teacher_forcing:
#         # Teacher forcing: Feed the target as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             loss += criterion(decoder_output, target_tensor[di])
#             decoder_input = target_tensor[di]  # Teacher forcing
#
#
#     else:
#         # Without teacher forcing: use its own predictions as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach()  # detach from history as input
#
#             loss += criterion(decoder_output, target_tensor[di])
#             if decoder_input.item() == EOS_token:
#                 break
#
#     # Backpropagate the loss and update the model weights
#     loss.backward()
#     encoder_optimizer.step()
#     style_embedder_optimizer.step()
#     decoder_optimizer.step()
#
#     return loss.item() / target_length
#
#


In [ ]:
# #Trainin
#
# teacher_forcing_ratio = 0.5
#
#
# def train(
#         input_tensor,
#         style_tensor,
#         target_tensor,
#
#         encoder,
#         style_embedder,
#         decoder,
#
#         encoder_optimizer,
#         style_embedder_optimizer,
#         decoder_optimizer,
#
#         criterion,
#         max_length=MAX_LENGTH):
#
#
#     encoder_hidden = encoder.initHidden()
#
#     encoder_optimizer.zero_grad()
#     decoder_optimizer.zero_grad()
#     style_embedder_optimizer.zero_grad()
#
#     input_length = input_tensor.size(0)
#     target_length = target_tensor.size(0)
#
#     encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
#
#     loss = 0
#
#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[ei], encoder_hidden)
#         encoder_outputs[ei] = encoder_output[0, 0]
#
#     # for ei in range(input_length):
#     #     encoder_output, encoder_hidden = encoder( input_tensor[ei], encoder_hidden)
#     #     encoder_outputs[ei] = encoder_output[0, 0]
#
#
#     decoder_input = torch.tensor([[SOS_token]], device=device)
#
#     # decoder_hidden = encoder_hidden
#
#     #calculate style embedding
#     style_embedding = style_embedder(style_tensor)
#
#     decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2) #TODO: concatenate style embedding
#
#     use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
#
#     # if use_teacher_forcing:
#     #     # Teacher forcing: Feed the target as the next input
#     #     for di in range(target_length):
#     #         decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
#     #         loss += criterion(decoder_output, target_tensor[di])
#     #         decoder_input = target_tensor[di]  # Teacher forcing
#     #
#     # else:
#     # # Without teacher forcing: use its own predictions as the next input
#     #     for di in range(target_length):
#     #         decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
#     #         topv, topi = decoder_output.topk(1)
#     #         decoder_input = topi.squeeze().detach()  # detach from history as input
#     #
#     #         loss += criterion(decoder_output, target_tensor[di])
#     #         if decoder_input.item() == EOS_token:
#     #             break
#     # #
#     if use_teacher_forcing:
#         # Teacher forcing: Feed the target as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             loss += criterion(decoder_output, target_tensor[di])
#             decoder_input = target_tensor[di]  # Teacher forcing
#
#     else:
#         # Without teacher forcing: use its own predictions as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach()  # detach from history as input
#
#             loss += criterion(decoder_output, target_tensor[di])
#             if decoder_input.item() == EOS_token:
#                 break
#
#     loss.backward()
#     encoder_optimizer.step()
#     style_embedder_optimizer.step()
#     decoder_optimizer.step()
#
#
#     return loss.item() / target_length

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(pairs, test_size=0.05, random_state=42)
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")
print(f"{train_data[0]}")
print(test_data[0])

Number of training examples: 14250
Number of testing examples: 750
[' dan jessi terancam kena pasal seksual celana dalam ', 'hyuna dan jessi terancam kena pasal seksual gara-gara umbar celana dalam ?', 1]
['bekasi gandeng 3 perusahaan olah sampah jadi listrik', 'bekasi gandeng 3 perusahaan olah sampah jadi listrik', 0]


In [ ]:
def trainIters(
        encoder,
        style_embedder,
        decoder,

        criterion,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,

        n_iters,
        print_every=1000,
        plot_every=100):

    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder.train()
    style_embedder.train()
    decoder.train()
    
    training_pairs = []
    for i in range(n_iters - 100):
      if i < len(train_data):
        print(i)
        pair = train_data[i]
        print(pair)
        if checkTensorExist(input_lang, pair[0]) == True and checkTensorExist(output_lang, pair[1]) == True:
          training_pairs.append(tensorsFromPair(pair))

    # training_pairs = [tensorsFromPair(random.choice(pairs))for i in range(n_iters)]
    # criterion = nn.NLLLoss()

    train_len = len(training_pairs)

    print("this is the real len of training pairs",train_len )

    ouput_loss_total = 0
    # for iter in range(1, n_iters + 1):
    for iter in range(1, train_len):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        style_tensor = training_pair[1]
        target_tensor = training_pair[2]

        loss = train(input_tensor, style_tensor, target_tensor, encoder, style_embedder,
                     decoder, encoder_optimizer, style_embedder_optimizer, decoder_optimizer, criterion)

        print_loss_total += loss
        plot_loss_total += loss
        ouput_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    # showPlot(plot_losses)
    return ouput_loss_total/n_iters


In [ ]:
def evaluating(
        encoder,
        style_embedder,
        decoder,

        criterion,
        n_iters,
        plot_every : int =10,
        max_length=MAX_LENGTH
):

    encoder.eval()
    style_embedder.eval()
    decoder.eval()

    plot_losses = []
    plot_loss_total = 0
    loss_total_output = 0

    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    # training_pairs = [tensorsFromPair(train_data[i])for i in range(n_iters)]

    training_pairs = []
    for i in range(n_iters - 100):
      if i < len(test_data):
        pair = test_data[i]
        if checkTensorExist(input_lang, pair[0]) == True and checkTensorExist(output_lang, pair[1]) == True:
          training_pairs.append(tensorsFromPair(pair))

    test_len = len(training_pairs)

    start: float = time.time()

    with torch.no_grad():
        for iter  in  range(1, test_len + 1):
            # taking the pair and extract the c, a, target
            training_pair = training_pairs[iter - 1]
            input_tensor = training_pair[0]
            style_tensor = training_pair[1]
            target_tensor = training_pair[2]

            encoder_hidden = encoder.initHidden()

            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            style_embedder_optimizer.zero_grad()


            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            # for ei in range(input_length):
            #     encoder_output, encoder_hidden = encoder(
            #         input_tensor[ei], encoder_hidden)
            #     encoder_outputs[ei] = encoder_output[0, 0]

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(
                    input_tensor[ei], encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]


            decoder_input = torch.tensor([[SOS_token]], device=device)

            #calculate style embedding
            style_embedding = style_embedder(style_tensor)

            decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2) #TODO: concatenate style embedding

            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            if use_teacher_forcing:
                # Teacher forcing: Feed the target as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    loss += criterion(decoder_output, target_tensor[di])
                    decoder_input = target_tensor[di]  # Teacher forcing

            else:
                # Without teacher forcing: use its own predictions as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += criterion(decoder_output, target_tensor[di])
                    if decoder_input.item() == EOS_token:
                        break

            # if use_teacher_forcing:
            #     # Teacher forcing: Feed the target as the next input
            #     for di in range(target_length):
            #         decoder_output, decoder_hidden, decoder_attention = decoder(
            #             decoder_input, decoder_hidden, encoder_outputs)
            #         loss += criterion(decoder_output, target_tensor[di])
            #         decoder_input = target_tensor[di]  # Teacher forcing
            #
            # else:
            #     # Without teacher forcing: use its own predictions as the next input
            #     for di in range(target_length):
            #         decoder_output, decoder_hidden, decoder_attention = decoder(
            #             decoder_input, decoder_hidden, encoder_outputs)
            #         topv, topi = decoder_output.topk(1)
            #         decoder_input = topi.squeeze().detach()  # detach from history as input
            #
            #         loss += criterion(decoder_output, target_tensor[di])
            #         if decoder_input.item() == EOS_token:
            #             break


            l = loss.item() / target_length

            loss_total_output += l
            plot_loss_total += l

    return loss_total_output/ n_iters

In [ ]:
# word_vec_size = 128
# word_vec_size = ft.vector_size

# hidden_size = 512
# style_vec_size = 128

# hidden_size = ft.vector_size

# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input ).to(device)
# decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words).to(device)

# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input, ENC_DROPOUT).to(device)
# decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output, DEC_DROPOUT).to(device)

# decoder1 = AttnDecoderRNN(hidden_size + style_vec_size, output_lang.n_words, embedding_matrix_ft_output,  dropout_p=0.1).to(device)


# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
# decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)
#
# style_embedder1 = StyleEmbedder(2, style_vec_size).to(device)

In [ ]:
# # word_vec_size = 128
# word_vec_size = ft.vector_size

# hidden_size = 512
# style_vec_size = 128

# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
# decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)

# style_embedder1 = StyleEmbedder(2, style_vec_size).to(device)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
INPUT_MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/deleteOnly_input_lang_dec_15_dec_16.pt"
OUTPUT_MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/deleteOnly_output_lang_dec_15_dec_16.pt"

torch.save(input_lang, INPUT_MAIN_PATH)
torch.save(output_lang, OUTPUT_MAIN_PATH)


# RUN on Dec 15 [Succes, cannot load]

In [ ]:
n_t = len(train_data)
n_e = len(test_data)
N_EPOCHS = 60
best_valid_loss = float('inf')

# learning_rate=0.1
# learning_rate = 0.001

# encoder_optimizer = optim.Adadelta(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.Adadelta(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.Adadelta(style_embedder1.parameters(), lr=learning_rate)

# encoder_optimizer = optim.Adam(encoder1.parameters(),lr=learning_rate)
# decoder_optimizer = optim.Adam(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=learning_rate)

# encoder_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.SGD(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.SGD(style_embedder1.parameters(), lr=learning_rate)
#
# encoder_optimizer = optim.RMSprop(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.RMSprop(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.RMSprop(style_embedder1.parameters(), lr=learning_rate)


encoder_learning_rate = 0.001
decoder_learning_rate = 0.001
style_embedder_learning_rate = 0.001

# Use different optimization algorithms for each optimizer
encoder_optimizer = optim.Adam(encoder1.parameters(), lr=encoder_learning_rate)
decoder_optimizer = optim.SGD(decoder1.parameters(), lr=decoder_learning_rate)
style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=style_embedder_learning_rate)

# Use a learning rate schedule to adjust the learning rates during training
encoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(encoder_optimizer, 'min')
decoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(decoder_optimizer, 'min')
style_embedder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(style_embedder_optimizer, 'min')

criterion1 = nn.CrossEntropyLoss()

TRAINING_DATASET_SIZE = n_t
EVALUATION_DATASET_SIZE = n_e

# PATH = "delete_only_model.pt"
PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_model_dec_15_512_scheduler_dec16.pt"
MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/"

# criterion1 = nn.NLLLoss()


plot_train_losses = []
plot_eval_losses = []
perplexity_train = []
perplexity_eval = []

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model_dec15_dec16.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model_dec15_dec16.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model_dec15_dec16.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(encoder1.state_dict(), 'encoder1-model.pt')
    #     torch.save(style_embedder1.state_dict(), 'style_embedder1-model.pt')
    #     torch.save(decoder1.state_dict(), 'decoder1-model.pt')

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)


    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))



    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")

showPlot(plot_train_losses)
showPlot(plot_eval_losses)

0m 6s (- 7m 16s) (200 1%) 8.6478
0m 10s (- 5m 50s) (400 2%) 9.2417
0m 13s (- 5m 8s) (600 4%) 7.2635
0m 17s (- 4m 47s) (800 5%) 6.9646
0m 20s (- 4m 30s) (1000 7%) 6.7520
0m 23s (- 4m 19s) (1200 8%) 6.6687
0m 27s (- 4m 10s) (1400 9%) 6.9004
0m 30s (- 4m 4s) (1600 11%) 6.9170
0m 34s (- 3m 57s) (1800 12%) 6.9823
0m 37s (- 3m 51s) (2000 14%) 7.0161
0m 41s (- 3m 46s) (2200 15%) 6.9681
0m 45s (- 3m 42s) (2400 16%) 7.1517
0m 48s (- 3m 38s) (2600 18%) 6.9658
0m 52s (- 3m 33s) (2800 19%) 7.0211
0m 55s (- 3m 29s) (3000 21%) 7.0002
0m 59s (- 3m 24s) (3200 22%) 7.0634
1m 2s (- 3m 20s) (3400 23%) 7.0021
1m 6s (- 3m 16s) (3600 25%) 7.1457
1m 9s (- 3m 12s) (3800 26%) 7.0116
1m 13s (- 3m 8s) (4000 28%) 7.0334
1m 17s (- 3m 4s) (4200 29%) 7.0725
1m 20s (- 3m 1s) (4400 30%) 7.0344
1m 24s (- 2m 57s) (4600 32%) 7.0728
1m 28s (- 2m 53s) (4800 33%) 7.0079
1m 31s (- 2m 50s) (5000 35%) 7.0088
1m 35s (- 2m 46s) (5200 36%) 6.9696
1m 39s (- 2m 42s) (5400 37%) 6.9178
1m 43s (- 2m 39s) (5600 39%) 6.9696
1m 46s (- 2m

<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Epoch: 11 | Time: 4m 44s
	Train Loss: 4.508 | Train PPL:  90.734
	 Val. Loss: 4.530 |  Val. PPL:  92.743
Done
0m 5s (- 6m 37s) (200 1%) 4.4046
0m 9s (- 5m 36s) (400 2%) 4.5659
0m 13s (- 5m 11s) (600 4%) 4.4433
0m 17s (- 4m 58s) (800 5%) 4.4649
0m 21s (- 4m 46s) (1000 7%) 4.3598
0m 25s (- 4m 37s) (1200 8%) 4.3056
0m 29s (- 4m 28s) (1400 9%) 4.4839
0m 33s (- 4m 23s) (1600 11%) 4.4413
0m 37s (- 4m 17s) (1800 12%) 4.4543
0m 41s (- 4m 12s) (2000 14%) 4.3648
0m 45s (- 4m 7s) (2200 15%) 4.3223
0m 49s (- 4m 2s) (2400 16%) 4.4389
0m 53s (- 3m 58s) (2600 18%) 4.4040
0m 56s (- 3m 52s) (2800 19%) 4.4437
1m 0s (- 3m 48s) (3000 21%) 4.3020
1m 4s (- 3m 43s) (3200 22%) 4.4181
1m 8s (- 3m 39s) (3400 23%) 4.4284
1m 12s (- 3m 34s) (3600 25%) 4.5407
1m 16s (- 3m 30s) (3800 26%) 4.3685
1m 20s (- 3m 26s) (4000 28%) 4.3517
1m 24s (- 3m 22s) (4200 29%) 4.5055
1m 28s (- 3m 17s) (4400 30%) 4.5029
1m 32s (- 3m 13s) (4600 32%) 4.4621
1m 36s (- 3m 9s) (4800 33%) 4.3400
1m 40s (- 3m 5s) (5000 35%) 4.4088
1m 44s (- 

<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Epoch: 12 | Time: 4m 46s
	Train Loss: 4.325 | Train PPL:  75.541
	 Val. Loss: 4.365 |  Val. PPL:  78.662
Done
0m 5s (- 6m 32s) (200 1%) 4.2470
0m 9s (- 5m 39s) (400 2%) 4.3917
0m 13s (- 5m 13s) (600 4%) 4.2483
0m 17s (- 4m 58s) (800 5%) 4.3070
0m 21s (- 4m 46s) (1000 7%) 4.1686
0m 25s (- 4m 37s) (1200 8%) 4.1201
0m 29s (- 4m 30s) (1400 9%) 4.3554
0m 33s (- 4m 24s) (1600 11%) 4.2515
0m 37s (- 4m 18s) (1800 12%) 4.2694
0m 41s (- 4m 12s) (2000 14%) 4.1814
0m 45s (- 4m 6s) (2200 15%) 4.1316
0m 48s (- 4m 1s) (2400 16%) 4.2519
0m 52s (- 3m 57s) (2600 18%) 4.1991
0m 56s (- 3m 52s) (2800 19%) 4.2576
1m 0s (- 3m 47s) (3000 21%) 4.1339
1m 4s (- 3m 42s) (3200 22%) 4.2608
1m 8s (- 3m 38s) (3400 23%) 4.2494
1m 12s (- 3m 33s) (3600 25%) 4.3063
1m 16s (- 3m 29s) (3800 26%) 4.2041
1m 20s (- 3m 25s) (4000 28%) 4.1814
1m 24s (- 3m 21s) (4200 29%) 4.2793
1m 27s (- 3m 16s) (4400 30%) 4.2528
1m 31s (- 3m 12s) (4600 32%) 4.2528
1m 35s (- 3m 8s) (4800 33%) 4.1644
1m 39s (- 3m 4s) (5000 35%) 4.1840
1m 43s (- 

...

<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Epoch: 59 | Time: 4m 45s
	Train Loss: 0.998 | Train PPL:   2.713
	 Val. Loss: 1.255 |  Val. PPL:   3.508
Done
0m 5s (- 6m 28s) (200 1%) 1.0433
0m 9s (- 5m 33s) (400 2%) 1.0232
0m 13s (- 5m 12s) (600 4%) 0.9716
0m 17s (- 4m 59s) (800 5%) 1.0232
0m 21s (- 4m 48s) (1000 7%) 1.0759
0m 25s (- 4m 38s) (1200 8%) 0.9395
0m 29s (- 4m 31s) (1400 9%) 0.9525
0m 33s (- 4m 24s) (1600 11%) 0.9927
0m 37s (- 4m 17s) (1800 12%) 0.9970
0m 41s (- 4m 11s) (2000 14%) 1.0622
0m 44s (- 4m 6s) (2200 15%) 0.9978
0m 48s (- 4m 1s) (2400 16%) 0.9868
0m 52s (- 3m 57s) (2600 18%) 0.9969
0m 56s (- 3m 52s) (2800 19%) 0.9897
1m 0s (- 3m 47s) (3000 21%) 0.9391
1m 4s (- 3m 42s) (3200 22%) 0.9632
1m 8s (- 3m 37s) (3400 23%) 0.9366
1m 12s (- 3m 33s) (3600 25%) 0.9505
1m 15s (- 3m 28s) (3800 26%) 0.9419
1m 19s (- 3m 24s) (4000 28%) 0.9623
1m 23s (- 3m 20s) (4200 29%) 0.9967
1m 27s (- 3m 16s) (4400 30%) 1.0390
1m 31s (- 3m 12s) (4600 32%) 1.0290
1m 35s (- 3m 8s) (4800 33%) 1.0081
1m 39s (- 3m 4s) (5000 35%) 0.9752
1m 43s (- 

<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Epoch: 60 | Time: 4m 44s
	Train Loss: 0.979 | Train PPL:   2.662
	 Val. Loss: 1.285 |  Val. PPL:   3.616
Done


<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


In [ ]:
print("plot_train_losses")
print(plot_train_losses)

print("plot_eval_losses")
print(plot_eval_losses )

print("perplexity_train" )
print(perplexity_train )

print("perplexity_eval" )
print(perplexity_eval )

plot_train_losses
[7.0068375797428315, 6.711939109351545, 6.3819218540777385, 6.070325797484908, 5.802758224489428, 5.550262481662367, 5.319225785529268, 5.097967935732267, 4.895242840364587, 4.691900554125262, 4.507927497683239, 4.324673452791832, 4.1467414624728685, 3.9727625588477737, 3.800212928348247, 3.646541178987324, 3.4943733370519743, 3.342832477782037, 3.1955323690904653, 3.06227495466009, 2.9189269525607653, 2.788744931157142, 2.6655220147859358, 2.546800798743993, 2.4479015544227325, 2.3510183505053592, 2.2460031195628556, 2.1573554818720835, 2.0737894242537322, 1.998940833387803, 1.922329451606338, 1.8498076454589152, 1.793496498334929, 1.748635940618143, 1.690382966996678, 1.6404289885632037, 1.5946128828022876, 1.5508718158867563, 1.5136017727072943, 1.472120677430797, 1.4267114914170893, 1.3783504490626488, 1.3554325070010709, 1.3238898469935736, 1.285095679892091, 1.2500155464121094, 1.2358404322707348, 1.2048651031130815, 1.1874662274728949, 1.1558067784660149, 1.144

In [ ]:
N_EPOCHS_2 = 10
for epoch in range(N_EPOCHS_2):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)


    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model_dec15.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model_dec15.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model_dec15.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(encoder1.state_dict(), 'encoder1-model.pt')
    #     torch.save(style_embedder1.state_dict(), 'style_embedder1-model.pt')
    #     torch.save(decoder1.state_dict(), 'decoder1-model.pt')




    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")

0m 5s (- 6m 36s) (200 1%) 1.0794
0m 9s (- 5m 27s) (400 2%) 0.9989
0m 13s (- 5m 9s) (600 4%) 0.9865
0m 17s (- 4m 54s) (800 5%) 0.9451
0m 21s (- 4m 42s) (1000 7%) 0.9864
0m 25s (- 4m 34s) (1200 8%) 0.8855
0m 29s (- 4m 27s) (1400 9%) 0.9215
0m 33s (- 4m 21s) (1600 11%) 0.9495
0m 36s (- 4m 15s) (1800 12%) 0.9132
0m 40s (- 4m 9s) (2000 14%) 0.9693
0m 44s (- 4m 4s) (2200 15%) 0.9361
0m 48s (- 3m 59s) (2400 16%) 0.9264
0m 52s (- 3m 54s) (2600 18%) 0.9156
0m 56s (- 3m 49s) (2800 19%) 0.9490
1m 0s (- 3m 45s) (3000 21%) 0.9158
1m 3s (- 3m 40s) (3200 22%) 0.9098
1m 7s (- 3m 36s) (3400 23%) 0.9133
1m 11s (- 3m 31s) (3600 25%) 0.9192
1m 15s (- 3m 27s) (3800 26%) 0.9141
1m 19s (- 3m 23s) (4000 28%) 0.8871
1m 23s (- 3m 19s) (4200 29%) 0.9389
1m 27s (- 3m 15s) (4400 30%) 0.9604
1m 31s (- 3m 10s) (4600 32%) 0.9722
1m 34s (- 3m 6s) (4800 33%) 0.9672
1m 38s (- 3m 2s) (5000 35%) 0.8948
1m 42s (- 2m 58s) (5200 36%) 0.9110
1m 46s (- 2m 54s) (5400 37%) 1.0115
1m 50s (- 2m 51s) (5600 39%) 0.9431
1m 54s (- 2m 

<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Epoch: 01 | Time: 4m 45s
	Train Loss: 0.945 | Train PPL:   2.572
	 Val. Loss: 1.243 |  Val. PPL:   3.464
Done
0m 5s (- 6m 46s) (200 1%) 1.0336
0m 9s (- 5m 41s) (400 2%) 1.0179
0m 13s (- 5m 16s) (600 4%) 0.9816
0m 17s (- 5m 1s) (800 5%) 0.9629
0m 21s (- 4m 49s) (1000 7%) 0.9872
0m 25s (- 4m 40s) (1200 8%) 0.8544
0m 29s (- 4m 33s) (1400 9%) 0.9342
0m 33s (- 4m 27s) (1600 11%) 0.9702
0m 37s (- 4m 21s) (1800 12%) 0.9329
0m 41s (- 4m 15s) (2000 14%) 0.9824
0m 45s (- 4m 10s) (2200 15%) 0.9503
0m 49s (- 4m 6s) (2400 16%) 0.9558
0m 53s (- 4m 1s) (2600 18%) 0.9185
0m 57s (- 3m 56s) (2800 19%) 0.9826
1m 1s (- 3m 51s) (3000 21%) 0.9390
1m 5s (- 3m 46s) (3200 22%) 0.8948
1m 9s (- 3m 41s) (3400 23%) 0.9240
1m 13s (- 3m 37s) (3600 25%) 0.9226
1m 17s (- 3m 32s) (3800 26%) 0.9103
1m 21s (- 3m 28s) (4000 28%) 0.9396
1m 25s (- 3m 24s) (4200 29%) 0.9486
1m 29s (- 3m 20s) (4400 30%) 0.9518
1m 33s (- 3m 15s) (4600 32%) 0.9309
1m 37s (- 3m 11s) (4800 33%) 0.9434
1m 41s (- 3m 7s) (5000 35%) 0.9224
1m 45s (- 

<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Epoch: 02 | Time: 4m 48s
	Train Loss: 0.941 | Train PPL:   2.562
	 Val. Loss: 1.202 |  Val. PPL:   3.326
Done
0m 5s (- 6m 33s) (200 1%) 0.9810
0m 9s (- 5m 34s) (400 2%) 0.9875
0m 13s (- 5m 10s) (600 4%) 0.9492
0m 17s (- 4m 57s) (800 5%) 0.9423
0m 21s (- 4m 44s) (1000 7%) 0.9504
0m 25s (- 4m 34s) (1200 8%) 0.9230
0m 29s (- 4m 28s) (1400 9%) 0.9454
0m 33s (- 4m 22s) (1600 11%) 0.9179
0m 36s (- 4m 15s) (1800 12%) 0.9154
0m 40s (- 4m 9s) (2000 14%) 0.9749
0m 44s (- 4m 4s) (2200 15%) 0.9195
0m 48s (- 3m 59s) (2400 16%) 0.9028
0m 52s (- 3m 55s) (2600 18%) 0.9127
0m 56s (- 3m 50s) (2800 19%) 0.9305
1m 0s (- 3m 45s) (3000 21%) 0.9334
1m 4s (- 3m 41s) (3200 22%) 0.8835
1m 7s (- 3m 36s) (3400 23%) 0.8771
1m 11s (- 3m 32s) (3600 25%) 0.8893
1m 15s (- 3m 28s) (3800 26%) 0.8907
1m 19s (- 3m 24s) (4000 28%) 0.9073
1m 23s (- 3m 20s) (4200 29%) 0.9388
1m 27s (- 3m 15s) (4400 30%) 0.9395
1m 31s (- 3m 11s) (4600 32%) 0.9425
1m 35s (- 3m 7s) (4800 33%) 0.9049
1m 39s (- 3m 3s) (5000 35%) 0.9537
1m 42s (- 

...

Epoch: 08 | Time: 4m 41s
	Train Loss: 0.874 | Train PPL:   2.395
	 Val. Loss: 1.143 |  Val. PPL:   3.136
Done
0m 5s (- 6m 28s) (200 1%) 0.9326
0m 9s (- 5m 35s) (400 2%) 0.9477
0m 13s (- 5m 8s) (600 4%) 0.8988
0m 17s (- 4m 55s) (800 5%) 0.9118
0m 21s (- 4m 42s) (1000 7%) 0.8797
0m 25s (- 4m 34s) (1200 8%) 0.8515
0m 29s (- 4m 26s) (1400 9%) 0.8716
0m 32s (- 4m 20s) (1600 11%) 0.8845
0m 36s (- 4m 13s) (1800 12%) 0.8607
0m 40s (- 4m 7s) (2000 14%) 0.8883
0m 44s (- 4m 2s) (2200 15%) 0.8641
0m 48s (- 3m 57s) (2400 16%) 0.8831
0m 52s (- 3m 53s) (2600 18%) 0.8944
0m 55s (- 3m 48s) (2800 19%) 0.8896
0m 59s (- 3m 44s) (3000 21%) 0.8721
1m 3s (- 3m 39s) (3200 22%) 0.9057
1m 7s (- 3m 34s) (3400 23%) 0.8623
1m 10s (- 3m 30s) (3600 25%) 0.8310
1m 14s (- 3m 25s) (3800 26%) 0.8536
1m 18s (- 3m 21s) (4000 28%) 0.8325
1m 22s (- 3m 17s) (4200 29%) 0.8550
1m 26s (- 3m 13s) (4400 30%) 0.8450
1m 30s (- 3m 9s) (4600 32%) 0.8473
1m 34s (- 3m 5s) (4800 33%) 0.8155
1m 37s (- 3m 1s) (5000 35%) 0.7913
1m 41s (- 2

<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Epoch: 09 | Time: 4m 41s
	Train Loss: 0.847 | Train PPL:   2.332
	 Val. Loss: 1.126 |  Val. PPL:   3.084
Done
0m 5s (- 6m 28s) (200 1%) 0.9142
0m 9s (- 5m 32s) (400 2%) 0.9037
0m 13s (- 5m 8s) (600 4%) 0.8734
0m 17s (- 4m 54s) (800 5%) 0.8533
0m 21s (- 4m 42s) (1000 7%) 0.8745
0m 25s (- 4m 33s) (1200 8%) 0.7987
0m 29s (- 4m 26s) (1400 9%) 0.8166
0m 32s (- 4m 20s) (1600 11%) 0.8074
0m 36s (- 4m 13s) (1800 12%) 0.8214
0m 40s (- 4m 7s) (2000 14%) 0.8748
0m 44s (- 4m 2s) (2200 15%) 0.8839
0m 48s (- 3m 58s) (2400 16%) 0.8140
0m 52s (- 3m 53s) (2600 18%) 0.8367
0m 55s (- 3m 48s) (2800 19%) 0.8285
0m 59s (- 3m 44s) (3000 21%) 0.8186
1m 3s (- 3m 39s) (3200 22%) 0.8215
1m 7s (- 3m 34s) (3400 23%) 0.7981
1m 11s (- 3m 30s) (3600 25%) 0.8294
1m 14s (- 3m 26s) (3800 26%) 0.8640
1m 18s (- 3m 21s) (4000 28%) 0.7990
1m 22s (- 3m 17s) (4200 29%) 0.8448
1m 26s (- 3m 13s) (4400 30%) 0.8624
1m 30s (- 3m 9s) (4600 32%) 0.8412
1m 34s (- 3m 5s) (4800 33%) 0.8342
1m 38s (- 3m 1s) (5000 35%) 0.7983
1m 41s (- 2

<ipython-input-59-e0689a3a6113>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-59-e0689a3a6113>:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Epoch: 10 | Time: 4m 41s
	Train Loss: 0.841 | Train PPL:   2.318
	 Val. Loss: 1.085 |  Val. PPL:   2.959
Done


# RERUN ON 16 DEC ; SAME MODEL; 

In [ ]:
n_t = len(train_data)
n_e = len(test_data)
N_EPOCHS = 60
best_valid_loss = float('inf')

In [ ]:
INPUT_MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/deleteOnly_input_lang_dec_15_dec_16.pt"
OUTPUT_MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/deleteOnly_output_lang_dec_15_dec_16.pt"

torch.save(input_lang, INPUT_MAIN_PATH)
torch.save(output_lang, OUTPUT_MAIN_PATH)


In [ ]:

# learning_rate=0.1
# learning_rate = 0.001

# encoder_optimizer = optim.Adadelta(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.Adadelta(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.Adadelta(style_embedder1.parameters(), lr=learning_rate)

# encoder_optimizer = optim.Adam(encoder1.parameters(),lr=learning_rate)
# decoder_optimizer = optim.Adam(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=learning_rate)

# encoder_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.SGD(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.SGD(style_embedder1.parameters(), lr=learning_rate)
#
# encoder_optimizer = optim.RMSprop(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.RMSprop(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.RMSprop(style_embedder1.parameters(), lr=learning_rate)


encoder_learning_rate = 0.001
decoder_learning_rate = 0.001
style_embedder_learning_rate = 0.001

# Use different optimization algorithms for each optimizer
encoder_optimizer = optim.Adam(encoder1.parameters(), lr=encoder_learning_rate)
decoder_optimizer = optim.SGD(decoder1.parameters(), lr=decoder_learning_rate)
style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=style_embedder_learning_rate)

# Use a learning rate schedule to adjust the learning rates during training
encoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(encoder_optimizer, 'min')
decoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(decoder_optimizer, 'min')
style_embedder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(style_embedder_optimizer, 'min')

criterion1 = nn.CrossEntropyLoss()

TRAINING_DATASET_SIZE = n_t
EVALUATION_DATASET_SIZE = n_e

# PATH = "delete_only_model.pt"
PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_model_dec_15_512_scheduler_dec16.pt"
MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/"
ACCURACY_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_losses_dec_15_512_dec16.pt"

# criterion1 = nn.NLLLoss()


plot_train_losses = []
plot_eval_losses = []
perplexity_train = []
perplexity_eval = []

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model_dec15_dec16.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model_dec15_dec16.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model_dec15_dec16.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(encoder1.state_dict(), 'encoder1-model.pt')
    #     torch.save(style_embedder1.state_dict(), 'style_embedder1-model.pt')
    #     torch.save(decoder1.state_dict(), 'decoder1-model.pt')

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)

    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))

    torch.save({
      'plot_train_losses' : plot_train_losses,
      'plot_eval_losses':  plot_eval_losses,
      'perplexity_train': perplexity_train, 
      "perplexity_eval" : perplexity_eval, 
    }, ACCURACY_PATH )



    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")

showPlot(plot_train_losses)
showPlot(plot_eval_losses)

0m 8s (- 9m 43s) (200 1%) 9.4824
0m 14s (- 8m 7s) (400 2%) 9.3985
0m 18s (- 7m 7s) (600 4%) 6.9807
0m 23s (- 6m 34s) (800 5%) 6.4722
0m 28s (- 6m 15s) (1000 7%) 6.9115
0m 32s (- 5m 58s) (1200 8%) 6.5170
0m 38s (- 5m 49s) (1400 9%) 6.9517
0m 43s (- 5m 40s) (1600 11%) 6.8422
0m 48s (- 5m 32s) (1800 12%) 6.7993
0m 52s (- 5m 24s) (2000 14%) 6.8909
0m 57s (- 5m 17s) (2200 15%) 6.8768
1m 3s (- 5m 12s) (2400 16%) 6.8462
1m 8s (- 5m 7s) (2600 18%) 6.9674
1m 13s (- 5m 1s) (2800 19%) 6.9779
1m 19s (- 4m 56s) (3000 21%) 6.9785
1m 24s (- 4m 50s) (3200 22%) 7.1435
1m 29s (- 4m 44s) (3400 23%) 7.0855
1m 34s (- 4m 39s) (3600 25%) 7.1393
1m 39s (- 4m 34s) (3800 26%) 7.0499
1m 45s (- 4m 29s) (4000 28%) 7.0578
1m 50s (- 4m 24s) (4200 29%) 7.0351
1m 55s (- 4m 19s) (4400 30%) 7.0084
2m 1s (- 4m 14s) (4600 32%) 7.1289
2m 6s (- 4m 9s) (4800 33%) 7.0157
2m 11s (- 4m 3s) (5000 35%) 7.0939
2m 17s (- 3m 58s) (5200 36%) 6.9733
2m 22s (- 3m 53s) (5400 37%) 6.9364
2m 28s (- 3m 48s) (5600 39%) 6.9460
2m 33s (- 3m 4

In [ ]:
print("plot_train_losses")
print(plot_train_losses)

print("plot_eval_losses")
print(plot_eval_losses )

print("perplexity_train" )
print(perplexity_train )

print("perplexity_eval" )
print(perplexity_eval )

plot_train_losses
[7.005529635069425, 6.699265200084709, 6.381756733422616, 6.069959938892099, 5.790355372701792, 5.543380059807032, 5.314447089941987, 5.1036299328474835, 4.897923136668892, 4.703650494363527, 4.515460452806608, 4.338235304809176, 4.159913261000685, 3.9875564099402516, 3.8176959763198544, 3.66523023134419, 3.5021149827089877, 3.3645800648132744, 3.215164983438294, 3.0856280115588604, 2.9586023433819846, 2.8294666336084617, 2.7142906032198653, 2.593688281236336, 2.5070213422797827, 2.3959467670875836, 2.307352648218475, 2.224968113296315, 2.1382526824792376, 2.074252433482152, 2.0141103065638783, 1.9421391865373947, 1.876762465914096, 1.8327792398523242, 1.7852132337370066, 1.7162480753643083, 1.6791864725334353, 1.6438323615821455, 1.593639300909478, 1.5644508693512131, 1.5297762982118046, 1.4962815332905384, 1.479152984033667, 1.4578008764622925, 1.4403702184839522, 1.418800639187358, 1.3961254198501503, 1.3739730995625903, 1.341365585916353, 1.3085475692005082, 1.298

In [ ]:
N_EPOCHS_2 = 10
for epoch in range(N_EPOCHS_2):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model_dec15_dec16.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model_dec15_dec16.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model_dec15_dec16.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(encoder1.state_dict(), 'encoder1-model.pt')
    #     torch.save(style_embedder1.state_dict(), 'style_embedder1-model.pt')
    #     torch.save(decoder1.state_dict(), 'decoder1-model.pt')

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)

    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))

    torch.save({
      'plot_train_losses' : plot_train_losses,
      'plot_eval_losses':  plot_eval_losses,
      'perplexity_train': perplexity_train, 
      "perplexity_eval" : perplexity_eval, 
    }, ACCURACY_PATH )



    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")


0m 7s (- 8m 21s) (200 1%) 1.2917
0m 12s (- 7m 27s) (400 2%) 1.2467
0m 18s (- 7m 6s) (600 4%) 1.2469
0m 24s (- 6m 53s) (800 5%) 1.2343
0m 30s (- 6m 41s) (1000 7%) 1.2087
0m 36s (- 6m 33s) (1200 8%) 1.1806
0m 41s (- 6m 25s) (1400 9%) 1.1722
0m 47s (- 6m 16s) (1600 11%) 1.2554
0m 53s (- 6m 8s) (1800 12%) 1.2016
0m 58s (- 5m 59s) (2000 14%) 1.2505
1m 4s (- 5m 53s) (2200 15%) 1.2098
1m 10s (- 5m 46s) (2400 16%) 1.2484
1m 15s (- 5m 39s) (2600 18%) 1.2315
1m 21s (- 5m 33s) (2800 19%) 1.2618
1m 27s (- 5m 27s) (3000 21%) 1.2471
1m 32s (- 5m 20s) (3200 22%) 1.1655
1m 38s (- 5m 13s) (3400 23%) 1.1509
1m 43s (- 5m 7s) (3600 25%) 1.2077
1m 49s (- 5m 1s) (3800 26%) 1.1906
1m 55s (- 4m 55s) (4000 28%) 1.0943
2m 0s (- 4m 49s) (4200 29%) 1.2101
2m 6s (- 4m 43s) (4400 30%) 1.2074
2m 12s (- 4m 37s) (4600 32%) 1.1953
2m 18s (- 4m 31s) (4800 33%) 1.2225
2m 23s (- 4m 25s) (5000 35%) 1.1620
2m 29s (- 4m 20s) (5200 36%) 1.1848
2m 35s (- 4m 14s) (5400 37%) 1.3280
2m 40s (- 4m 8s) (5600 39%) 1.2826
2m 46s (- 4m

In [ ]:
N_EPOCHS_2 = 20
for epoch in range(N_EPOCHS_2):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model_dec15_dec16.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model_dec15_dec16.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model_dec15_dec16.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(encoder1.state_dict(), 'encoder1-model.pt')
    #     torch.save(style_embedder1.state_dict(), 'style_embedder1-model.pt')
    #     torch.save(decoder1.state_dict(), 'decoder1-model.pt')

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)

    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))

    torch.save({
      'plot_train_losses' : plot_train_losses,
      'plot_eval_losses':  plot_eval_losses,
      'perplexity_train': perplexity_train, 
      "perplexity_eval" : perplexity_eval, 
    }, ACCURACY_PATH )



    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")


0m 7s (- 8m 20s) (200 1%) 1.1958
0m 12s (- 7m 28s) (400 2%) 1.1733
0m 18s (- 7m 3s) (600 4%) 1.2101
0m 24s (- 6m 51s) (800 5%) 1.1824
0m 30s (- 6m 39s) (1000 7%) 1.1055
0m 35s (- 6m 27s) (1200 8%) 1.0718
0m 41s (- 6m 20s) (1400 9%) 1.0995
0m 47s (- 6m 13s) (1600 11%) 1.1685
0m 52s (- 6m 4s) (1800 12%) 1.1035
0m 58s (- 5m 57s) (2000 14%) 1.1578
1m 3s (- 5m 50s) (2200 15%) 1.1197
1m 9s (- 5m 44s) (2400 16%) 1.1949
1m 15s (- 5m 38s) (2600 18%) 1.1634
1m 21s (- 5m 31s) (2800 19%) 1.0997
1m 26s (- 5m 26s) (3000 21%) 1.1122
1m 32s (- 5m 18s) (3200 22%) 1.0931
1m 37s (- 5m 12s) (3400 23%) 1.0586
1m 43s (- 5m 6s) (3600 25%) 1.1291
1m 49s (- 5m 0s) (3800 26%) 1.1097
1m 55s (- 4m 54s) (4000 28%) 1.0585
2m 0s (- 4m 48s) (4200 29%) 1.0659
2m 6s (- 4m 42s) (4400 30%) 1.0793
2m 12s (- 4m 36s) (4600 32%) 1.1266
2m 17s (- 4m 31s) (4800 33%) 1.0880
2m 23s (- 4m 25s) (5000 35%) 1.1017
2m 29s (- 4m 19s) (5200 36%) 1.0785
2m 34s (- 4m 13s) (5400 37%) 1.0645
2m 40s (- 4m 8s) (5600 39%) 1.1403
2m 46s (- 4m 

In [ ]:
N_EPOCHS_2 = 30
for epoch in range(N_EPOCHS_2):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model_dec15_dec16.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model_dec15_dec16.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model_dec15_dec16.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(encoder1.state_dict(), 'encoder1-model.pt')
    #     torch.save(style_embedder1.state_dict(), 'style_embedder1-model.pt')
    #     torch.save(decoder1.state_dict(), 'decoder1-model.pt')

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)

    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))

    torch.save({
      'plot_train_losses' : plot_train_losses,
      'plot_eval_losses':  plot_eval_losses,
      'perplexity_train': perplexity_train, 
      "perplexity_eval" : perplexity_eval, 
    }, ACCURACY_PATH )



    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")


0m 7s (- 8m 20s) (200 1%) 1.0631
0m 12s (- 7m 27s) (400 2%) 1.0494
0m 18s (- 7m 6s) (600 4%) 1.0851
0m 24s (- 6m 58s) (800 5%) 1.0440
0m 30s (- 6m 43s) (1000 7%) 1.0214
0m 36s (- 6m 32s) (1200 8%) 0.9395
0m 41s (- 6m 25s) (1400 9%) 0.9587
0m 47s (- 6m 17s) (1600 11%) 1.0832
0m 53s (- 6m 9s) (1800 12%) 0.9979
0m 58s (- 6m 0s) (2000 14%) 0.9871
1m 4s (- 5m 54s) (2200 15%) 0.9920
1m 10s (- 5m 47s) (2400 16%) 0.9894
1m 16s (- 5m 41s) (2600 18%) 1.0163
1m 21s (- 5m 34s) (2800 19%) 1.0149
1m 27s (- 5m 28s) (3000 21%) 1.0388
1m 33s (- 5m 22s) (3200 22%) 1.0136
1m 38s (- 5m 15s) (3400 23%) 0.9753
1m 44s (- 5m 8s) (3600 25%) 1.0183
1m 50s (- 5m 3s) (3800 26%) 1.0405
1m 56s (- 4m 57s) (4000 28%) 0.9851
2m 1s (- 4m 50s) (4200 29%) 0.9644
2m 7s (- 4m 44s) (4400 30%) 0.9886
2m 12s (- 4m 38s) (4600 32%) 1.0262
2m 18s (- 4m 33s) (4800 33%) 1.0226
2m 24s (- 4m 27s) (5000 35%) 0.9681
2m 30s (- 4m 21s) (5200 36%) 0.9913
2m 36s (- 4m 15s) (5400 37%) 1.0465
2m 41s (- 4m 10s) (5600 39%) 1.0360
2m 47s (- 4m

# Load the previous model [FAILED]


In [ ]:
INPUT_MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/deleteOnly_input_lang_dec_15_dec_16.pt"
OUTPUT_MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/deleteOnly_output_lang_dec_15_dec_16.pt"
PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_model_dec_15_512_scheduler_dec16.pt"
MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/"
ACCURACY_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_losses_dec_15_512_dec16.pt"


In [ ]:
# Resources: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Had to modify & adapt most of the code in the tutorial since this isn't translation & data preprocessing is different

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

input_lang = Lang("input")
output_lang = Lang("output")

input_lang = torch.load(INPUT_MAIN_PATH)
output_lang = torch.load(OUTPUT_MAIN_PATH)

In [ ]:
word_vec_size = ft.vector_size

hidden_size = 512
style_vec_size = 128

encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)

style_embedder1 = StyleEmbedder(2, style_vec_size).to(device)

In [ ]:
n_t = len(train_data)
n_e = len(test_data)
N_EPOCHS = 60
best_valid_loss = float('inf')

# # learning_rate=0.1
# # learning_rate = 0.001

# # encoder_optimizer = optim.Adadelta(encoder1.parameters(), lr=learning_rate)
# # decoder_optimizer = optim.Adadelta(decoder1.parameters(), lr=learning_rate)
# # style_embedder_optimizer = optim.Adadelta(style_embedder1.parameters(), lr=learning_rate)

# # encoder_optimizer = optim.Adam(encoder1.parameters(),lr=learning_rate)
# # decoder_optimizer = optim.Adam(decoder1.parameters(), lr=learning_rate)
# # style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=learning_rate)

# # encoder_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate)
# # decoder_optimizer = optim.SGD(decoder1.parameters(), lr=learning_rate)
# # style_embedder_optimizer = optim.SGD(style_embedder1.parameters(), lr=learning_rate)
# #
# # encoder_optimizer = optim.RMSprop(encoder1.parameters(), lr=learning_rate)
# # decoder_optimizer = optim.RMSprop(decoder1.parameters(), lr=learning_rate)
# # style_embedder_optimizer = optim.RMSprop(style_embedder1.parameters(), lr=learning_rate)


encoder_learning_rate = 0.001
decoder_learning_rate = 0.001
style_embedder_learning_rate = 0.001

# Use different optimization algorithms for each optimizer
encoder_optimizer = optim.Adam(encoder1.parameters(), lr=encoder_learning_rate)
decoder_optimizer = optim.SGD(decoder1.parameters(), lr=decoder_learning_rate)
style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=style_embedder_learning_rate)

# Use a learning rate schedule to adjust the learning rates during training
encoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(encoder_optimizer, 'min')
decoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(decoder_optimizer, 'min')
style_embedder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(style_embedder_optimizer, 'min')

criterion1 = nn.CrossEntropyLoss()



# PATH = "delete_only_model.pt"
# PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_model_dec_15_512_scheduler.pt"
# MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/"

# criterion1 = nn.NLLLoss()


plot_train_losses = []
plot_eval_losses = []
perplexity_train = []
perplexity_eval = []


In [ ]:
import torch

# Create the models and optimizers
# encoder1 = Encoder()
# style_embedder1 = StyleEmbedder()
# decoder1 = Decoder()
# encoder_optimizer = optim.Adam(encoder1.parameters())
# decoder_optimizer = optim.Adam(decoder1.parameters())
# style_embedder_optimizer = optim.Adam(style_embedder1.parameters())

# Load the saved state dictionaries and optimizer state dictionaries
checkpoint = torch.load(PATH)
encoder1.load_state_dict(checkpoint['encoder1_state_dict'])
style_embedder1.load_state_dict(checkpoint['style_embedder1-model'])
decoder1.load_state_dict(checkpoint['decoder1_state_dict'])
encoder_optimizer.load_state_dict(checkpoint['encoder1_optimizer_state_dict'])
decoder_optimizer.load_state_dict(checkpoint['decoder1_optimizer_state_dict'])
style_embedder_optimizer.load_state_dict(checkpoint['style_embedder1_optimizer_state_dict'])

# # Save the epoch and loss values
# epoch = checkpoint['epoch']
# train_loss = checkpoint['loss']


In [ ]:
    # torch.save({
    #   'plot_train_losses' : plot_train_losses,
    #   'plot_eval_losses':  plot_eval_losses,
    #   'perplexity_train': perplexity_train, 
    #   "perplexity_eval" : perplexity_eval, 
    # }, ACCURACY_PATH )

checkpoint_metrics = torch.load(ACCURACY_PATH)
plot_train_losses =  checkpoint_metrics['plot_train_losses']
plot_eval_losses = checkpoint_metrics['plot_eval_losses']
perplexity_train = checkpoint_metrics['perplexity_train']
perplexity_eval = checkpoint_metrics['perplexity_eval']

In [ ]:
# plot_eval_losses

In [ ]:
TRAINING_DATASET_SIZE = n_t - 1000
EVALUATION_DATASET_SIZE = n_e - 20

In [ ]:
print(TRAINING_DATASET_SIZE)
print(EVALUATION_DATASET_SIZE)

In [ ]:
# ACCURACY_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_losses_dec_15_512.pt"
N_EPOCHS_2 = 10
for epoch in range(N_EPOCHS_2):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)

    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))


    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model_dec15.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model_dec15.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model_dec15.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(encoder1.state_dict(), 'encoder1-model.pt')
    #     torch.save(style_embedder1.state_dict(), 'style_embedder1-model.pt')
    #     torch.save(decoder1.state_dict(), 'decoder1-model.pt')



    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")



Streaming output truncated to the last 5000 lines.
[' kesehatan besok - kesehatan jumat 20 september 2019 virgo & aquarius diet', 'zodiak kesehatan besok - ramalan zodiak kesehatan jumat 20 september 2019 virgo & aquarius diet', 1]
10683
['komentar marquez dan rossi soal insiden sesi kualifikasi gp san marino', 'komentar marquez dan rossi soal insiden sesi kualifikasi gp san marino', 1]
10684
['netanyahu janji caplok lebih banyak wilayah tepi barat jika menang pemilu', 'netanyahu janji caplok lebih banyak wilayah tepi barat jika menang pemilu', 0]
10685
['agensi bicara tentang kembalinya mina di ', 'agensi bicara tentang kembalinya mina di comeback twice', 0]
10686
['mampukah indonesia jadi negara maju di 2045 ', 'mampukah indonesia jadi negara maju di 2045 ?', 1]
10687
['ajak warga cinta lingkungan , iwan fals gelar konser karya semesta', 'ajak warga cinta lingkungan , iwan fals gelar konser karya semesta', 0]
10688
['buronan hingga sempat rekam , 4 korban selamat kecelakaan nganjuk-m

IndexError: ignored

# Create Clickbait title for Nonclickbait dataset

In [ ]:
encoder = encoder1
decoder = decoder1
style_embedder = style_embedder1

In [ ]:
# input_lang, output_lang, pairs, pairs_nonclick, pairs_nonclick  = prepareData()

In [ ]:
def encoderStep_deleteOnly(encoder, style_embedder, sentence, style, max_length=MAX_LENGTH):
    input_tensor = tensorFromSentence(input_lang, sentence)
    style_tensor = tensorFromStyle(style)

    input_length = input_tensor.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] += encoder_output[0, 0]
    style_embedding = style_embedder(style_tensor)
    return torch.cat((encoder_hidden, style_embedding), 2)



ALPHA = 0.7

class Node:
    def __init__(self, p_t, i, d_hidden, lvl, node_parent=None):
        self.p_t = p_t
        self.i = i
        self.d_hidden = d_hidden
        self.lvl = lvl
        self.node_parent = node_parent
        
        self.p_sentence = self.sentenceProb()
    
    def prepareToDecode(self):
        d_in = self.i.squeeze()#.detatch()
        return d_in, self.d_hidden
    
    def sentenceProb(self):
        if self.node_parent == None:
            return self.p_t
        return self.node_parent.sentenceProb() + self.p_t #addition because of log
    
    def normProb(self):
        #Normalised probability - deprecated
#         return 1/((self.lvl+1)**ALPHA) * self.sentenceProb()
        #perplexity
        return torch.exp(self.sentenceProb()) ** (-1/(self.lvl+1))
    
    def getToken(self):
        return output_lang.index2word[self.i.item()]
    
    def getTokens(self):
        token = [self.getToken()]
#         print(token)
        if(self.node_parent == None):
            return token
        else:
             return self.node_parent.getTokens() + token
#         return tokens
    def getSentence(self):
        return ' '.join(self.getTokens())
# Normalize 1/num words^alpha (alpha = 0.7)

In [ ]:
BEAM_WIDTH = 10

def evaluate_deleteOnly(encoder, style_embedder, decoder, sentence, style, max_length=MAX_LENGTH):
    with torch.no_grad():    
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoderStep_deleteOnly(encoder, style_embedder, sentence, style)
        decoded_words = []

        nodes = []
        finished_nodes = []
        for di in range(max_length):
            if di == 0:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                top_ps, top_is = decoder_output.data.topk(BEAM_WIDTH)

                top_ps = top_ps.view(-1)
                top_is = top_is.view(-1)

                for index in range(len(top_is)):
                    p_t = top_ps[index]
                    i = top_is[index]
                    nodes.append(Node(p_t, i, decoder_hidden, di))

            else:
                prev_nodes = [x for x in nodes if x.lvl == di-1]
                prev_nodes = sorted(prev_nodes, key=lambda x: x.p_sentence.item(), reverse=True)
                prev_nodes = prev_nodes[:BEAM_WIDTH]

                nodes = []
                for node in prev_nodes:
                    decoder_output, decoder_hidden = decoder(*node.prepareToDecode())
                    top_ps, top_is = decoder_output.data.topk(BEAM_WIDTH)
                    top_ps = top_ps.view(-1)
                    top_is = top_is.view(-1)    
                    
                    for index in range(len(top_is)):
                        p_t = top_ps[index]
                        i = top_is[index]
                        child_node = Node(p_t, i, decoder_hidden, di, node)
                        
                        if i.item() == EOS_token:
                            finished_nodes.append(child_node)
                        else:
                            nodes.append(child_node)
                            
        final_node = sorted(finished_nodes, key=lambda x: x.p_sentence.item(), reverse=True)[0]
        return final_node.getSentence()
# evaluate_deleteOnly(encoder1, style_embedder1, decoder1, "the food", 1)

In [ ]:
def evaluateRandomly(encoder, style_embedder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        print('style: ', pair[2])
        
        
        output_words = evaluate_deleteOnly(encoder, style_embedder, decoder, pair[0], pair[2])
        output_sentence = ' '.join(output_words)
        
        print('<', output_sentence)
        print('')

def evaluateRandomly2(encoder, style_embedder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        print('style: ', pair[2])
        
        
        output_words = evaluate_deleteOnly(encoder, style_embedder, decoder, pair[0], 0)
        output_sentence = ' '.join(output_words)
        
        print('<', output_sentence)
        print('')

In [ ]:
evaluateRandomly2(encoder1, style_embedder1, decoder1)

> gocar resmi beroperasi di bandara soekarno-hatta
= gocar resmi beroperasi di bandara soekarno-hatta
style:  0
< g o c a r   p r o y e k   b e r o p e r a s i   d i   b a n d a r a   s o e k a r n o - h a t t a   E O S

>  motor nyebrang sungai ala flying fox , colek kementerian pupr
= viral motor nyebrang sungai ala flying fox , netizen colek kementerian pupr
style:  1
< b o t o l   m o t o r   s u n g a i   s u n g a i   m i n y a k   d e n g a n   m i n y a k   ,   n e t i z e n   ,   k u d u s   E O S

> mensesneg siapkan draf , jokowi bakal terbitkan perppu untuk kpk 
= mensesneg siapkan draf , jokowi bakal terbitkan perppu untuk kpk ?
style:  1
< r k u h p   g e l a r   i n v e s t a s i   ,   j o k o w i   b a k a l   b u a t   b a h a s   r u u   k p k   d i s a h k a n   E O S

> total , jumlah tersangka rusuh di papua ada 87 orang
= total , jumlah tersangka rusuh di papua ada 87 orang
style:  0
< t o t a l   ,   d u a   m a h a s i s w a   r u s u h   d i   p a p u a   j i k

# Retrained again at 17 December

In [169]:
print("plot_train_losses")
print(plot_train_losses)

print("plot_eval_losses")
print(plot_eval_losses )

print("perplexity_train" )
print(perplexity_train )

print("perplexity_eval" )
print(perplexity_eval )

plot_train_losses
[7.005529635069425, 6.699265200084709, 6.381756733422616, 6.069959938892099, 5.790355372701792, 5.543380059807032, 5.314447089941987, 5.1036299328474835, 4.897923136668892, 4.703650494363527, 4.515460452806608, 4.338235304809176, 4.159913261000685, 3.9875564099402516, 3.8176959763198544, 3.66523023134419, 3.5021149827089877, 3.3645800648132744, 3.215164983438294, 3.0856280115588604, 2.9586023433819846, 2.8294666336084617, 2.7142906032198653, 2.593688281236336, 2.5070213422797827, 2.3959467670875836, 2.307352648218475, 2.224968113296315, 2.1382526824792376, 2.074252433482152, 2.0141103065638783, 1.9421391865373947, 1.876762465914096, 1.8327792398523242, 1.7852132337370066, 1.7162480753643083, 1.6791864725334353, 1.6438323615821455, 1.593639300909478, 1.5644508693512131, 1.5297762982118046, 1.4962815332905384, 1.479152984033667, 1.4578008764622925, 1.4403702184839522, 1.418800639187358, 1.3961254198501503, 1.3739730995625903, 1.341365585916353, 1.3085475692005082, 1.298

In [ ]:
PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_model_dec_16_512_scheduler_dec17.pt"
MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/"
ACCURACY_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_losses_dec_16_512_dec17.pt"

In [168]:
N_EPOCHS_2 = 20
for epoch in range(N_EPOCHS_2):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model_dec16_dec17.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model_dec16_dec17.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model_dec16_dec17.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)

    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))

    torch.save({
      'plot_train_losses' : plot_train_losses,
      'plot_eval_losses':  plot_eval_losses,
      'perplexity_train': perplexity_train, 
      "perplexity_eval" : perplexity_eval, 
    }, ACCURACY_PATH )


    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")


Streaming output truncated to the last 5000 lines.
['netanyahu janji caplok lebih banyak wilayah tepi barat jika menang pemilu', 'netanyahu janji caplok lebih banyak wilayah tepi barat jika menang pemilu', 0]
10685
['agensi bicara tentang kembalinya mina di ', 'agensi bicara tentang kembalinya mina di comeback twice', 0]
10686
['mampukah indonesia jadi negara maju di 2045 ', 'mampukah indonesia jadi negara maju di 2045 ?', 1]
10687
['ajak warga cinta lingkungan , iwan fals gelar konser karya semesta', 'ajak warga cinta lingkungan , iwan fals gelar konser karya semesta', 0]
10688
['buronan hingga sempat rekam , 4 korban selamat kecelakaan nganjuk-madiun', 'buronan hingga sempat rekam video , 4 fakta mengejutkan korban selamat kecelakaan nganjuk-madiun', 1]
10689
['pesona jennie blackpink yang memikat , pernah nyanyi bareng seungri', 'pesona jennie blackpink yang memikat , pernah nyanyi bareng seungri', 0]
10690
['sudah cukupkah kandungan protein rambutmu ', 'sudah cukupkah kandungan pro

# BLEU Evaluation Metrics